# EcoImpact AI - Model

  ## Import Libraries 

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,mean_absolute_percentage_error,classification_report,confusion_matrix
import pickle
import warnings
warnings.filterwarnings('ignore')

## Load Dataset

In [43]:
df = pd.read_csv(r'C:\Users\HP\Desktop\fyp\docs\Ecoimpact -AI\dataset\processed\ecoimpact_complete_dataset.csv')

## Exploring Dataset and Finding Missing values

In [44]:
df.shape

(2808, 25)

In [45]:
df.columns.tolist()

['Unique ID',
 'Instrument name',
 'Type',
 'Status',
 'Jurisdiction',
 'Region',
 'Income group',
 'Year',
 'Emission_Coverage_%',
 'Carbon_Price_USD',
 'Revenue_Million_USD',
 'Coal per capita (kWh)',
 'Oil per capita (kWh)',
 'Gas per capita (kWh)',
 'Nuclear per capita (kWh - equivalent)',
 'Hydro per capita (kWh - equivalent)',
 'Wind per capita (kWh - equivalent)',
 'Solar per capita (kWh - equivalent)',
 'Other renewables per capita (kWh - equivalent)',
 'Total_Energy_per_capita',
 'Fossil_Fuel_Dependency_%',
 'Population',
 'Population_Log',
 'Annual_CO2_emissions',
 'GDP']

In [46]:
df.head(3)

,Unique ID,Instrument name,Type,Status,Jurisdiction,Region,Income group,Year,Emission_Coverage_%,Carbon_Price_USD,...,Hydro per capita (kWh - equivalent),Wind per capita (kWh - equivalent),Solar per capita (kWh - equivalent),Other renewables per capita (kWh - equivalent),Total_Energy_per_capita,Fossil_Fuel_Dependency_%,Population,Population_Log,Annual_CO2_emissions,GDP
0,Tax_AL,Albania carbon tax,Carbon tax,Implemented,Albania,Europe & Central Asia,Upper middle income,1990,0.0,NaN,...,2475.4448,2.986670,0.047724,81.68200,48572.274394,86.738952,3277965.0,15.002734,5520602.0,1.281962e+10
1,Tax_CA_Alberta,Alberta carbon tax,Carbon tax,Abolished,Alberta,North America,High income,1990,0.0,NaN,...,29555.9260,0.191839,0.000000,444.63837,106191.335709,64.927726,27789437.0,17.140167,458017860.0,1.051178e+12
2,ETS_CA_Alberta,Alberta TIER,ETS,Implemented,Alberta,North America,High income,1990,0.0,NaN,...,29555.9260,0.191839,0.000000,444.63837,106191.335709,64.927726,27789437.0,17.140167,458017860.0,1.051178e+12


In [47]:
df['is_active'] = (
      (df['Emission_Coverage_%'] > 0) |
      (df['Carbon_Price_USD'] > 0) |
      (df['Revenue_Million_USD'] > 0)
  )

In [48]:
df_active = df[df['is_active']].copy()

In [49]:
print(f"Total rows: {len(df)}")
print(f"Active rows: {len(df_active)}")
print(f"Inactive rows: {len(df) - len(df_active)}")

Total rows: 2808
Active rows: 917
Inactive rows: 1891


Training on active policies only to match prediction distribution

In [50]:
feature_cols = ['Type', 'Region', 'Income group', 'Year','Emission_Coverage_%', 'Carbon_Price_USD', 'Revenue_Million_USD','Fossil_Fuel_Dependency_%', 'Population_Log']

 ##### Model Features (How They're Obtained)
                                                                                                                        
  **USER INPUTS (what user provides):**

  1. **Country** → Backend maps to Region + Income group + Fossil_Fuel_% + Population_Log
  2. **Type** → User selects: Carbon tax or ETS
  3. **Carbon_Price_USD** → User inputs via slider ($0-200/ton)
  4. **Year** → System uses current year (2026) or user-selected future year

  **DERIVED FEATURES (backend lookup from country):**

  **Region** (mapped from country)
  - Pakistan → East Asia & Pacific
  - Germany → Europe & Central Asia
  - Captures regional policy patterns and institutional capacity

  **Income group** (mapped from country)
  - Pakistan → Upper middle income
  - Germany → High income
  - World Bank classification, indicates economic development level

  **Fossil_Fuel_Dependency_%** (looked up from energy dataset)
  - Pakistan → 81.2%
  - Germany → 65%
  - Reflects country's energy structure and carbon pricing scope

  **Population_Log** (looked up from population dataset)
  - Pakistan → 19.33 (log of 247.5 million)
  - Germany → 18.28 (log of 83.2 million)
  - Scales revenue predictions by country size

  ##### Target Variables

  **Emission_Coverage_%** - Coverage Model predicts this

  **Revenue_Million_USD** - Revenue Model predicts this

  ##### Data Flow

  User inputs Pakistan + Carbon Tax + $15/ton → Backend looks up Pakistan's Region, Income, Fossil_Fuel_%, 
  Population_Log → Models predict Coverage% and Revenue

In [51]:
print("Missing values in key features:")
print(df_active[feature_cols].isnull().sum())

Missing values in key features:
Type                          0
Region                        0
Income group                  0
Year                          0
Emission_Coverage_%           0
Carbon_Price_USD             98
Revenue_Million_USD         196
Fossil_Fuel_Dependency_%     67
Population_Log               86
dtype: int64


In [52]:
print("Missing Fossil_Fuel_Dependency_% by Year:")
missing_fossil = df_active[df_active['Fossil_Fuel_Dependency_%'].isnull()]                                            
print(missing_fossil['Year'].value_counts().sort_index())

Missing Fossil_Fuel_Dependency_% by Year:
Year
2025    67
Name: count, dtype: int64


In [53]:
print("Missing Population_Log by Year:")
missing_pop = df_active[df_active['Population_Log'].isnull()]
print(missing_pop['Year'].value_counts().sort_index())


Missing Population_Log by Year:
Year
2005     1
2006     1
2007     1
2008     1
2009     1
2010     1
2011     1
2012     1
2013     1
2014     1
2015     1
2016     1
2017     1
2018     1
2019     1
2020     1
2021     1
2022     1
2023     1
2025    67
Name: count, dtype: int64


Strategy:
  - Fill EU population by summing 27 member states (same as GDP in Phase 1)
  - Coverage Model: Use up to 2024 (Fossil_Fuel_% available)
  - Revenue Model: Use up to 2023 (Population_Log available)

In [54]:
eu27_countries = [
      'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',                                                 
      'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
      'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
      'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
      'Slovenia', 'Spain', 'Sweden'
  ]

In [55]:
df_full = pd.read_csv(r'C:\Users\HP\Desktop\fyp\docs\Ecoimpact -AI\dataset\processed\ecoimpact_complete_dataset.csv')

In [56]:


for year in range(2005, 2024):
    year_data = df_full[(df_full['Year'] == year) & (df_full['Jurisdiction'].isin(eu27_countries))]
    eu_pop = year_data['Population'].sum()
    if eu_pop > 0:
        mask = (df_active['Jurisdiction'] == 'EU') & (df_active['Year'] == year)
        df_active.loc[mask, 'Population'] = eu_pop
        df_active.loc[mask, 'Population_Log'] = np.log(eu_pop)

eu_rows_after = len(df_active[(df_active['Jurisdiction'] == 'EU') & (df_active['Population_Log'].notna())])        
print(f"EU rows recovered with population: {eu_rows_after}")
print(f"\nTotal Revenue Model rows: {len(df_active)}")
print(f"Samples per feature: {len(df_active)} / 7 = {len(df_active)//7}")

EU rows recovered with population: 20

Total Revenue Model rows: 917
Samples per feature: 917 / 7 = 131


In [57]:
print("Missing Carbon_Price_USD by Year:")
missing_price = df_active[df_active['Carbon_Price_USD'].isnull()]
print(missing_price['Year'].value_counts().sort_index())

Missing Carbon_Price_USD by Year:
Year
1990    2
1991    4
1992    5
1993    5
1994    2
1995    2
1996    3
1997    4
1998    3
1999    1
2000    1
2001    1
2002    1
2003    1
2004    1
2005    1
2006    1
2008    2
2009    1
2010    1
2011    2
2012    2
2013    3
2014    4
2015    4
2016    2
2017    3
2018    4
2019    9
2020    7
2021    4
2022    4
2023    6
2024    1
2025    1
Name: count, dtype: int64


In [58]:
print("Missing Revenue_Million_USD by Year:")
missing_revenue = df_active[df_active['Revenue_Million_USD'].isnull()]
print(missing_revenue['Year'].value_counts().sort_index())

Missing Revenue_Million_USD by Year:
Year
1990     1
1991     2
1992     3
1993     3
1994     2
1995     2
1996     3
1997     3
1998     3
1999     1
2000     1
2001     1
2002     1
2003     1
2004     1
2005     2
2006     2
2007     3
2008     5
2009     2
2010     3
2011     4
2012     4
2013     5
2014     5
2015     5
2016     2
2017     2
2018     8
2019     8
2020     5
2021     6
2022     7
2023    10
2024    13
2025    67
Name: count, dtype: int64


In [59]:
print("Data availability summary:")
print(df_active.groupby('Year').agg({
      'Fossil_Fuel_Dependency_%': lambda x: x.notna().sum(),
      'Population_Log': lambda x: x.notna().sum()
  }).tail(5))

Data availability summary:
      Fossil_Fuel_Dependency_%  Population_Log
Year                                          
2021                        66              66
2022                        70              70
2023                        73              73
2024                        70              70
2025                         0               0


## Coverage Model

In [60]:
coverage_features = ['Type', 'Region', 'Income group', 'Year', 'Fossil_Fuel_Dependency_%']
revenue_features = ['Type', 'Region', 'Income group', 'Year', 'Carbon_Price_USD',
                     'Fossil_Fuel_Dependency_%', 'Population_Log']

In [61]:
df_coverage = df_active[df_active['Year'] <= 2024].copy()

In [62]:
print("Missing values in key features in coverage model:")
print(df_coverage[coverage_features].isnull().sum())

Missing values in key features in coverage model:
Type                        0
Region                      0
Income group                0
Year                        0
Fossil_Fuel_Dependency_%    0
dtype: int64


In [63]:
print(f"Population_Log missing: {df_coverage['Population_Log'].isnull().sum()}")
print(f"Annual_CO2_emissions missing: {df_coverage['Annual_CO2_emissions'].isnull().sum()}")

Population_Log missing: 0
Annual_CO2_emissions missing: 0


In [64]:
available_data = df_coverage.dropna(subset=['Population_Log', 'Annual_CO2_emissions'])
                                                                                                                        
corr_pop = available_data[['Population_Log', 'Emission_Coverage_%']].corr().iloc[0,1]                                 
corr_co2 = available_data[['Annual_CO2_emissions', 'Emission_Coverage_%']].corr().iloc[0,1]
corr_fossil = available_data[['Fossil_Fuel_Dependency_%', 'Emission_Coverage_%']].corr().iloc[0,1]

print(f"Population_Log vs Coverage%: r = {corr_pop:.3f}")
print(f"Annual_CO2 vs Coverage%: r = {corr_co2:.3f}")
print(f"Fossil_Fuel_% vs Coverage%: r = {corr_fossil:.3f} (already included)")


Population_Log vs Coverage%: r = 0.271
Annual_CO2 vs Coverage%: r = 0.219
Fossil_Fuel_% vs Coverage%: r = 0.116 (already included)


Final feature set (7 features) based on correlation analysis:
  - Categorical: Type, Region, Income group
  - Numerical: Year, Fossil_Fuel_Dependency_%, Population_Log, Annual_CO2_emissions

In [65]:
coverage_features = ['Type', 'Region', 'Income group', 'Year','Fossil_Fuel_Dependency_%', 'Population_Log', 'Annual_CO2_emissions']

### Train Test Split

In [66]:
X_coverage = df_coverage[coverage_features]

In [67]:
y_coverage = df_coverage['Emission_Coverage_%']

In [68]:
X_train_cov, X_test_cov, y_train_cov, y_test_cov = train_test_split(
      X_coverage,
      y_coverage,
      test_size=0.2,
      random_state=42,
      stratify=df_coverage['Region']
  )

In [69]:
print(f"Training set: {len(X_train_cov)} rows ({len(X_train_cov)/len(X_coverage)*100:.1f}%)")
print(f"Test set: {len(X_test_cov)} rows ({len(X_test_cov)/len(X_coverage)*100:.1f}%)")

Training set: 680 rows (80.0%)
Test set: 170 rows (20.0%)


### Encode Categorical Features 

- Fit encoders on training data only to prevent data leakage.
- Categorical features: Type, Region, Income group

In [70]:
categorical_features = ['Type', 'Region', 'Income group']

In [71]:
encoders = {}
X_train_encoded = X_train_cov.copy()
X_test_encoded = X_test_cov.copy()

In [72]:
for feature in categorical_features:
    encoders[feature] = LabelEncoder()
    X_train_encoded[feature] = encoders[feature].fit_transform(X_train_cov[feature])
    X_test_encoded[feature] = encoders[feature].transform(X_test_cov[feature])

In [73]:
for feature in categorical_features:
    mapping=dict(zip(encoders[feature].classes_, encoders[feature].transform(encoders[feature].classes_)))
    print(f"\nEncoding for {feature}:")
    for k, v in mapping.items():
        print(f"  {k}: {v}")


Encoding for Type:
  Carbon tax: 0
  ETS: 1

Encoding for Region:
  East Asia & Pacific: 0
  Europe & Central Asia: 1
  Latin America & Caribbean: 2
  North America: 3
  Sub-Saharan Africa: 4

Encoding for Income group:
  High income: 0
  Upper middle income: 1


In [74]:
print(f"X_train_encoded shape: {X_train_encoded.shape}")
print(f"X_test_encoded shape: {X_test_encoded.shape}")

X_train_encoded shape: (680, 7)
X_test_encoded shape: (170, 7)


### Train Coverage Model

Training 5 algorithms to find best performer:
  1. Linear Regression (baseline)                                                                                         2. Ridge Regression (regularized linear)
  3. Random Forest (ensemble, non-linear)                                                                               
  4. Gradient Boosting (ensemble, sequential)
  5. Support Vector Regression (non-linear)

In [75]:
models = {
      'Linear Regression': LinearRegression(),
      'Ridge Regression': Ridge(alpha=1.0, random_state=42),
      'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
      'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
      'SVR': SVR(kernel='rbf')
  }

In [76]:
results = []
for name, model in models.items():
      cv_scores = cross_val_score(model, X_train_encoded, y_train_cov,
                                  cv=5, scoring='r2', n_jobs=-1)

      results.append({
          'Model': name,
          'Mean R²': cv_scores.mean(),
          'Std R²': cv_scores.std()
      })

      print(f"{name}:")
      print(f"  Mean R² (CV): {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

Linear Regression:
  Mean R² (CV): 0.1214 (+/- 0.0592)
Ridge Regression:
  Mean R² (CV): 0.1214 (+/- 0.0588)
Random Forest:
  Mean R² (CV): 0.5587 (+/- 0.3173)
Gradient Boosting:
  Mean R² (CV): 0.5598 (+/- 0.3253)
SVR:
  Mean R² (CV): -34.4748 (+/- 9.2162)


In [77]:
results_df = pd.DataFrame(results).sort_values('Mean R²', ascending=False)
print("Ranking:")
print(results_df.to_string(index=False))

Ranking:
            Model    Mean R²   Std R²
Gradient Boosting   0.559791 0.325298
    Random Forest   0.558704 0.317330
Linear Regression   0.121384 0.059170
 Ridge Regression   0.121352 0.058825
              SVR -34.474848 9.216178


##### Results Interpretation
                                          
  Cross-validation shows tree-based models (Gradient Boosting, Random Forest) significantly outperform linear models:
  - Gradient Boosting & Random Forest: R² ≈ 0.56 (explain 56% of coverage variance)                                       - Linear/Ridge: R² ≈ 0.12 (poor fit, relationship is non-linear)
  - SVR: Negative R² (catastrophic failure with default hyperparameters)                                                
  
  The 56% R² indicates moderate predictive power - policy coverage follows regional/economic patterns, but
  implementation quality and political factors introduce variability that cannot be captured by these features alone.   

  High standard deviation (0.32-0.33) suggests performance varies across regions/time periods.

In [78]:
### Hyperparameter Tuning for Gradient Boosting

In [79]:
param_grid = {
      'n_estimators': [100, 200, 300],
      'learning_rate': [0.01, 0.05, 0.1],
      'max_depth': [3, 5, 7],
      'min_samples_split': [2, 5, 10]
  }

In [80]:
gb_model = GradientBoostingRegressor(random_state=42)

In [81]:
grid_search = GridSearchCV(
        gb_model,
        param_grid,
        cv=5,
        scoring='r2',
        n_jobs=-1,
        verbose=1
    )
grid_search.fit(X_train_encoded, y_train_cov)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='r2', verbose=1)

In [82]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV R²: {grid_search.best_score_:.4f}")
print(f"Improvement: {grid_search.best_score_ - 0.559791:.4f}")

Best parameters: {'learning_rate': 0.01, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 300}
Best CV R²: 0.5702
Improvement: 0.0105


In [83]:
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train_encoded)

In [84]:
X_train_with_pred = X_train_cov.copy()
X_train_with_pred['Predicted_Coverage'] = y_train_pred
X_train_with_pred['Actual_Coverage'] = y_train_cov.values
X_train_with_pred['Error'] = X_train_with_pred['Actual_Coverage'] - X_train_with_pred['Predicted_Coverage']

In [85]:
for region in X_train_with_pred['Region'].unique():
      region_data = X_train_with_pred[X_train_with_pred['Region'] == region]
      r2 = r2_score(region_data['Actual_Coverage'], region_data['Predicted_Coverage'])
      print(f"{region}: R² = {r2:.3f} (n={len(region_data)})")

Europe & Central Asia: R² = 0.996 (n=365)
East Asia & Pacific: R² = 0.276 (n=141)
North America: R² = 0.167 (n=125)
Latin America & Caribbean: R² = 0.726 (n=44)
Sub-Saharan Africa: R² = 0.365 (n=5)


In [86]:
for ptype in X_train_with_pred['Type'].unique():
      type_data = X_train_with_pred[X_train_with_pred['Type'] == ptype]
      r2 = r2_score(type_data['Actual_Coverage'], type_data['Predicted_Coverage'])
      print(f"{ptype}: R² = {r2:.3f} (n={len(type_data)})")

Carbon tax: R² = 0.972 (n=412)
ETS: R² = 0.578 (n=268)


##### Diagnostic: Regional Performance Heterogeneity
                                                                                         - Model performance varies dramatically across regions and policy types:
                                                                                                                        
  **By Region:** 
  - Europe & Central Asia: R² = 0.996 (excellent, n=365)
  - Latin America & Caribbean: R² = 0.726 (good, n=44)
  - East Asia & Pacific: R² = 0.276 (poor, n=141)
  - North America: R² = 0.167 (poor, n=125)
  - Sub-Saharan Africa: R² = 0.365 (limited data, n=5)

  **By Policy Type:**
  - Carbon tax: R² = 0.972 (excellent, n=412)
  - ETS: R² = 0.578 (moderate, n=268)

  **Problem Identified:** The model captures European carbon tax patterns extremely well but struggles with North       
  American ETS and East Asian policies. This suggests **regional heterogeneity** - coverage drivers differ by region and
   policy type.

  **Solution:** Add interaction features to let the model learn region-specific and type-specific patterns.

### Feature Engineering
Adding interaction features to capture regional heterogeneity:
  1. Region × Type - Different policy types behave differently across regions
  2. Region × Year - Regional adoption trajectories differ over time
  3. Type × Fossil_Fuel_% - Policy effectiveness varies with energy structure

In [87]:
X_train_interact = X_train_encoded.copy()   
X_test_interact = X_test_encoded.copy()

In [88]:
X_train_interact['Region_x_Type'] = X_train_encoded['Region'].astype(str) + '_' + X_train_encoded['Type'].astype(str)   
X_test_interact['Region_x_Type'] = X_test_encoded['Region'].astype(str) + '_' + X_test_encoded['Type'].astype(str)

In [89]:
X_train_interact['Region_x_Year'] = X_train_encoded['Region'] * X_train_encoded['Year']
X_test_interact['Region_x_Year'] = X_test_encoded['Region'] * X_test_encoded['Year']

In [90]:
X_train_interact['Type_x_Fossil'] = X_train_encoded['Type'] * X_train_encoded['Fossil_Fuel_Dependency_%']
X_test_interact['Type_x_Fossil'] = X_test_encoded['Type'] * X_test_encoded['Fossil_Fuel_Dependency_%']

In [91]:
encoder_region_type = LabelEncoder()
X_train_interact['Region_x_Type'] = encoder_region_type.fit_transform(X_train_interact['Region_x_Type'])
X_test_interact['Region_x_Type'] = encoder_region_type.transform(X_test_interact['Region_x_Type'])

In [92]:
print(f"Original features: {X_train_encoded.shape[1]}")
print(f"With interactions: {X_train_interact.shape[1]}")
print(f"\nNew interaction features:")
print(f"  - Region × Type (categorical)")
print(f"  - Region × Year (numerical)")
print(f"  - Type × Fossil_Fuel_% (numerical)")

Original features: 7
With interactions: 10

New interaction features:
  - Region × Type (categorical)
  - Region × Year (numerical)
  - Type × Fossil_Fuel_% (numerical)


  Using best hyperparameters from previous tuning plus new interaction features.

In [93]:
gb_interact = GradientBoostingRegressor(
      learning_rate=0.01,
      max_depth=5,
      min_samples_split=5,
      n_estimators=300,
      random_state=42
  )

In [94]:
cv_scores_interact = cross_val_score(gb_interact, X_train_interact, y_train_cov,cv=5, scoring='r2', n_jobs=-1)

In [95]:
print(f"BEFORE interaction features: R² = 0.5702")
print(f"AFTER interaction features:  R² = {cv_scores_interact.mean():.4f} (+/- {cv_scores_interact.std():.4f})")      
print(f"\nImprovement: {cv_scores_interact.mean() - 0.5702:.4f}")

BEFORE interaction features: R² = 0.5702
AFTER interaction features:  R² = 0.5648 (+/- 0.3244)

Improvement: -0.0054


In [96]:
gb_base = GradientBoostingRegressor(
      learning_rate=0.01,
      max_depth=5,
      min_samples_split=5,
      n_estimators=300,
      random_state=42
  )

In [97]:
gb_base.fit(X_train_encoded, y_train_cov)
y_test_pred=gb_base.predict(X_test_encoded)

In [98]:
test_r2 = r2_score(y_test_cov, y_test_pred)
test_mae = mean_absolute_error(y_test_cov, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test_cov, y_test_pred))

In [99]:
print(f"Training R² (CV): 0.5702")
print(f"Test R²: {test_r2:.4f}")
print(f"Test MAE: {test_mae:.2f}%")
print(f"Test RMSE: {test_rmse:.2f}%")

Training R² (CV): 0.5702
Test R²: 0.1277
Test MAE: 0.00%
Test RMSE: 0.01%


In [100]:
print("DIAGNOSTIC:")                                                                                                  
print(f"\nActual values (y_test_cov):") 
print(f"  Min: {y_test_cov.min():.2f}")
print(f"  Max: {y_test_cov.max():.2f}")                                                                                 
print(f"  Mean: {y_test_cov.mean():.2f}")
print(f"  Std: {y_test_cov.std():.2f}")

DIAGNOSTIC:

Actual values (y_test_cov):
  Min: 0.00
  Max: 0.15
  Mean: 0.00
  Std: 0.01


In [101]:
print(f"\nPredicted values (y_test_pred):")
print(f"  Min: {y_test_pred.min():.2f}")
print(f"  Max: {y_test_pred.max():.2f}")
print(f"  Mean: {y_test_pred.mean():.2f}")
print(f"  Std: {y_test_pred.std():.2f}")


Predicted values (y_test_pred):
  Min: 0.00
  Max: 0.05
  Mean: 0.00
  Std: 0.01


In [102]:
print(f"\nFirst 10 comparisons:")
comparison = pd.DataFrame({
      'Actual': y_test_cov.values[:10],
      'Predicted': y_test_pred[:10],
      'Error': np.abs(y_test_cov.values[:10] - y_test_pred[:10])
  })
print(comparison)


First 10 comparisons:
     Actual  Predicted     Error
0  0.000031   0.000231  0.000200
1  0.000596   0.000639  0.000044
2  0.002104   0.002369  0.000265
3  0.000041   0.000244  0.000203
4  0.000271   0.001423  0.001151
5  0.000426   0.001984  0.001558
6  0.005892   0.005032  0.000860
7  0.001261   0.000920  0.000341
8  0.000571   0.000646  0.000076
9  0.005797   0.001984  0.003814


In [103]:
print("Test set regional breakdown:")
X_test_with_pred = X_test_cov.copy()
X_test_with_pred['Predicted_Coverage'] = y_test_pred
X_test_with_pred['Actual_Coverage'] = y_test_cov.values

Test set regional breakdown:


In [104]:
for region in X_test_with_pred['Region'].unique():
    region_data = X_test_with_pred[X_test_with_pred['Region'] == region]
    r2 = r2_score(region_data['Actual_Coverage'], region_data['Predicted_Coverage'])
    print(f"{region}: Test R² = {r2:.3f} (n={len(region_data)})")

Europe & Central Asia: Test R² = 0.995 (n=91)
North America: Test R² = 0.083 (n=31)
East Asia & Pacific: Test R² = -0.063 (n=36)
Latin America & Caribbean: Test R² = 0.371 (n=11)
Sub-Saharan Africa: Test R² = nan (n=1)


In [105]:
sample_weights = np.ones(len(X_train_encoded))
region_weights = {
      'Europe & Central Asia': 1.0,
      'North America': 3.0,
      'East Asia & Pacific': 3.0,
      'Latin America & Caribbean': 5.0,
      'Sub-Saharan Africa': 10.0
    }

In [106]:
for region, weight in region_weights.items():
      mask = X_train_cov['Region'] == region
      sample_weights[mask] = weight

In [107]:
for region in X_train_cov['Region'].unique():
      count = (X_train_cov['Region'] == region).sum()
      weight = region_weights[region]
      effective = count * weight
      print(f"{region}: {count} samples × {weight} weight = {effective:.0f} effective")

Europe & Central Asia: 365 samples × 1.0 weight = 365 effective
East Asia & Pacific: 141 samples × 3.0 weight = 423 effective
North America: 125 samples × 3.0 weight = 375 effective
Latin America & Caribbean: 44 samples × 5.0 weight = 220 effective
Sub-Saharan Africa: 5 samples × 10.0 weight = 50 effective


In [108]:
gb_weighted = GradientBoostingRegressor(
      learning_rate=0.01,
        max_depth=5,
        min_samples_split=5,
        n_estimators=300,
        random_state=42
    )

In [109]:
gb_weighted.fit(X_train_encoded, y_train_cov, sample_weight=sample_weights)
y_test_pred_weighted = gb_weighted.predict(X_test_encoded)

In [110]:
test_r2_weighted = r2_score(y_test_cov, y_test_pred_weighted)

In [111]:
print(f"BEFORE weighting: Test R² = 0.1277")
print(f"AFTER weighting:  Test R² = {test_r2_weighted:.4f}")
print(f"\nBy region:")
X_test_weighted = X_test_cov.copy()
X_test_weighted['Predicted'] = y_test_pred_weighted
X_test_weighted['Actual'] = y_test_cov.values
for region in X_test_weighted['Region'].unique():
      region_data = X_test_weighted[X_test_weighted['Region'] == region]
      r2 = r2_score(region_data['Actual'], region_data['Predicted'])
      print(f"{region}: R² = {r2:.3f}")

BEFORE weighting: Test R² = 0.1277
AFTER weighting:  Test R² = 0.1259

By region:
Europe & Central Asia: R² = 0.985
North America: R² = 0.116
East Asia & Pacific: R² = -0.063
Latin America & Caribbean: R² = 0.391
Sub-Saharan Africa: R² = nan


In [112]:
non_europe_train = X_train_cov['Region'] != 'Europe & Central Asia'
non_europe_test = X_test_cov['Region'] != 'Europe & Central Asia'

In [113]:
X_train_row = X_train_encoded[non_europe_train]
y_train_row = y_train_cov[non_europe_train]
X_test_row = X_test_encoded[non_europe_test]
y_test_row = y_test_cov[non_europe_test]

In [114]:
gb_row = GradientBoostingRegressor( learning_rate=0.05, max_depth=3, min_samples_split=10, n_estimators=200,
  random_state=42)
gb_row.fit(X_train_row, y_train_row)
y_pred_row = gb_row.predict(X_test_row)


In [115]:
row_r2 = r2_score(y_test_row, y_pred_row)
print(f"REST OF WORLD MODEL: Test R² = {row_r2:.4f} (n_train={len(X_train_row)}, n_test={len(X_test_row)})")
print(f"Samples per feature: {len(X_train_row)} / 7 = {len(X_train_row)//7}")

REST OF WORLD MODEL: Test R² = -0.0392 (n_train=315, n_test=79)
Samples per feature: 315 / 7 = 45


In [116]:
df_coverage['Coverage_Log'] = np.log1p(df_coverage['Emission_Coverage_%'])

In [117]:
coverage_features = ['Type', 'Region', 'Income group', 'Year', 'Fossil_Fuel_Dependency_%', 'Population_Log',
  'Annual_CO2_emissions']

In [118]:
X_coverage = df_coverage[coverage_features]
y_coverage_log = df_coverage['Coverage_Log']

In [119]:
X_train_cov, X_test_cov, y_train_cov_log, y_test_cov_log = train_test_split(
      X_coverage, y_coverage_log, test_size=0.2, random_state=42, stratify=df_coverage['Region']
  )

In [120]:
encoders_cov = {}
X_train_cov_enc = X_train_cov.copy()
X_test_cov_enc = X_test_cov.copy()

In [121]:
for feature in ['Type', 'Region', 'Income group']:
      encoders_cov[feature] = LabelEncoder()
      X_train_cov_enc[feature] = encoders_cov[feature].fit_transform(X_train_cov[feature])
      X_test_cov_enc[feature] = encoders_cov[feature].transform(X_test_cov[feature])

In [122]:
gb_cov_log = GradientBoostingRegressor(learning_rate=0.01, max_depth=5, min_samples_split=5, n_estimators=300,        
  random_state=42)
gb_cov_log.fit(X_train_cov_enc, y_train_cov_log)
y_pred_cov_log = gb_cov_log.predict(X_test_cov_enc)

In [123]:
coverage_log_r2 = r2_score(y_test_cov_log, y_pred_cov_log)
print(f"Coverage (log-transformed) Test R²: {coverage_log_r2:.4f}")

print(f"\nBy region:")
X_test_cov_results = X_test_cov.copy()
X_test_cov_results['Predicted_Log'] = y_pred_cov_log
X_test_cov_results['Actual_Log'] = y_test_cov_log.values

for region in X_test_cov_results['Region'].unique():
    region_data = X_test_cov_results[X_test_cov_results['Region'] == region]
    if len(region_data) > 5:
        r2 = r2_score(region_data['Actual_Log'], region_data['Predicted_Log'])
        print(f"{region}: R² = {r2:.3f} (n={len(region_data)})")

Coverage (log-transformed) Test R²: 0.1407

By region:
Europe & Central Asia: R² = 0.995 (n=91)
North America: R² = 0.084 (n=31)
East Asia & Pacific: R² = -0.060 (n=36)
Latin America & Caribbean: R² = 0.363 (n=11)


##### Attempt: Adding GDP to Coverage Model                                                                              
                                                                                                                        
  Hypothesis: GDP might improve non-Europe coverage predictions by capturing institutional capacity.
  GDP improved Revenue model (0.72 → 0.87), testing if it helps Coverage.

In [124]:
df_coverage_gdp = df_active[df_active['Year'] <= 2024].copy()

In [125]:
liechtenstein_gdp = {
      2008: 5.08e9, 2009: 4.50e9, 2010: 5.08e9, 2011: 5.74e9,
      2012: 5.46e9, 2013: 6.39e9, 2014: 6.66e9, 2015: 6.27e9,
      2016: 6.24e9, 2017: 6.47e9, 2018: 6.69e9, 2019: 6.44e9,
      2020: 6.41e9, 2021: 7.91e9, 2022: 7.38e9, 2023: 8.29e9
  }

In [126]:
for year, gdp in liechtenstein_gdp.items():
      mask = (df_coverage_gdp['Jurisdiction'] == 'Liechtenstein') & (df_coverage_gdp['Year'] == year)
      df_coverage_gdp.loc[mask, 'GDP'] = gdp

In [127]:
for jurisdiction in ['Albania', 'Montenegro', 'Poland', 'Portugal']:
      gdp_2023 = df_coverage_gdp[(df_coverage_gdp['Jurisdiction'] == jurisdiction) & (df_coverage_gdp['Year'] ==        
  2023)]['GDP'].values
      if len(gdp_2023) > 0:
          mask = (df_coverage_gdp['Jurisdiction'] == jurisdiction) & (df_coverage_gdp['Year'] == 2024)
          df_coverage_gdp.loc[mask, 'GDP'] = gdp_2023[0]

df_coverage_gdp = df_coverage_gdp.dropna(subset=['Fossil_Fuel_Dependency_%', 'Emission_Coverage_%', 'GDP',
'Population_Log', 'Annual_CO2_emissions'])
df_coverage_gdp['Coverage_Log'] = np.log1p(df_coverage_gdp['Emission_Coverage_%'])

In [128]:
print(f"Coverage dataset with GDP: {len(df_coverage_gdp)} rows")
print(f"GDP missing: {df_coverage_gdp['GDP'].isnull().sum()}")

Coverage dataset with GDP: 784 rows
GDP missing: 0


In [129]:
coverage_features_gdp = ['Type', 'Region', 'Income group', 'Year',                                                    
                           'Fossil_Fuel_Dependency_%', 'Population_Log', 'Annual_CO2_emissions', 'GDP']

In [130]:
X_coverage_gdp = df_coverage_gdp[coverage_features_gdp]
y_coverage_gdp = df_coverage_gdp['Coverage_Log']

In [131]:
X_train_cov_gdp, X_test_cov_gdp, y_train_cov_gdp, y_test_cov_gdp = train_test_split(
      X_coverage_gdp, y_coverage_gdp, test_size=0.2, random_state=42, stratify=df_coverage_gdp['Region']
  )

In [132]:
encoders_cov_gdp = {}
X_train_cov_gdp_enc = X_train_cov_gdp.copy()
X_test_cov_gdp_enc = X_test_cov_gdp.copy()

In [133]:
for feature in ['Type', 'Region', 'Income group']:
      encoders_cov_gdp[feature] = LabelEncoder()
      X_train_cov_gdp_enc[feature] = encoders_cov_gdp[feature].fit_transform(X_train_cov_gdp[feature])
      X_test_cov_gdp_enc[feature] = encoders_cov_gdp[feature].transform(X_test_cov_gdp[feature])

In [134]:
gb_cov_gdp = GradientBoostingRegressor(learning_rate=0.01, max_depth=5, min_samples_split=5,
                                          n_estimators=500, subsample=0.8, random_state=42)
gb_cov_gdp.fit(X_train_cov_gdp_enc, y_train_cov_gdp)
y_pred_cov_gdp = gb_cov_gdp.predict(X_test_cov_gdp_enc)

In [135]:
r2_cov_gdp = r2_score(y_test_cov_gdp, y_pred_cov_gdp)

In [136]:
print(f"Coverage WITHOUT GDP (7 features): Test R² = 0.1407")
print(f"Coverage WITH GDP (8 features):    Test R² = {r2_cov_gdp:.4f}")
print(f"Improvement: {r2_cov_gdp - 0.1407:.4f}")

Coverage WITHOUT GDP (7 features): Test R² = 0.1407
Coverage WITH GDP (8 features):    Test R² = 0.3949
Improvement: 0.2542


In [137]:
X_test_cov_gdp_results = X_test_cov_gdp.copy()
X_test_cov_gdp_results['Predicted'] = y_pred_cov_gdp
X_test_cov_gdp_results['Actual'] = y_test_cov_gdp.values

In [138]:
print(f"\nBy region:")
for region in ['Europe & Central Asia', 'North America', 'East Asia & Pacific', 'Latin America & Caribbean']:
      region_data = X_test_cov_gdp_results[X_test_cov_gdp_results['Region'] == region]
      if len(region_data) > 5:
          r2 = r2_score(region_data['Actual'], region_data['Predicted'])
          print(f"{region}: R² = {r2:.3f} (n={len(region_data)})")


By region:
Europe & Central Asia: R² = 0.998 (n=87)
North America: R² = -0.845 (n=28)
East Asia & Pacific: R² = -0.036 (n=32)
Latin America & Caribbean: R² = 0.059 (n=9)


CORRELATION ANALYSIS BY REGION

In [139]:
regions = ['Europe & Central Asia', 'East Asia & Pacific', 'North America', 'Latin America & Caribbean']

In [140]:
for region in regions:
      region_data = df_coverage_gdp[df_coverage_gdp['Region'] == region]

      if len(region_data) > 20:
          print(f"\n{region} (n={len(region_data)}):")
          print("-" * 60)

          corr_features = ['Year', 'Fossil_Fuel_Dependency_%', 'Population_Log',
                          'Annual_CO2_emissions', 'GDP', 'Emission_Coverage_%']

          correlations = region_data[corr_features].corr()['Emission_Coverage_%'].sort_values(ascending=False)

          for feature, corr_val in correlations.items():
              if feature != 'Emission_Coverage_%':
                  print(f"  {feature:30s}: r = {corr_val:6.3f}")


Europe & Central Asia (n=433):
------------------------------------------------------------
  Annual_CO2_emissions          : r =  0.988
  GDP                           : r =  0.932
  Population_Log                : r =  0.516
  Fossil_Fuel_Dependency_%      : r =  0.110
  Year                          : r = -0.007

East Asia & Pacific (n=160):
------------------------------------------------------------
  Year                          : r =  0.144
  GDP                           : r =  0.069
  Population_Log                : r =  0.055
  Fossil_Fuel_Dependency_%      : r =  0.051
  Annual_CO2_emissions          : r =  0.032

North America (n=139):
------------------------------------------------------------
  Fossil_Fuel_Dependency_%      : r =  0.334
  Annual_CO2_emissions          : r =  0.330
  Population_Log                : r =  0.318
  GDP                           : r =  0.316
  Year                          : r = -0.274

Latin America & Caribbean (n=47):
---------------------

CRITICAL FINDING:                                                                                                        
  Europe:                                                                                                               
  - Annual_CO2_emissions: r = 0.988 (near perfect)                                                                      
  - GDP: r = 0.932 (extremely strong)
  - Features STRONGLY predict coverage

  East Asia:
  - All features: r < 0.15 (essentially ZERO correlation)
  - NO features predict coverage
  - Coverage is random from feature perspective

  North America & Latin America:
  - Weak correlations (r = 0.3-0.4)
  - Some signal but not strong

  ---
  WHAT THIS MEANS:

  European coverage is driven by measurable factors:
  - Economic size (GDP)
  - Emission scale (Annual_CO2)
  - Our features capture these → R² = 0.998 ✓

  Asian coverage is driven by UNMEASURABLE factors:
  - Political will, institutional capacity, governance quality
  - Our features don't capture these → R² = -0.036 ✗

  THIS IS NOT YOUR FAILURE - IT'S A DATA LIMITATION

  Regional correlation analysis reveals fundamental heterogeneity in coverage drivers. European coverage correlates    
  strongly with CO2 emissions (r=0.988) and GDP (r=0.932), enabling excellent predictions (R²=0.998). However, East     
  Asian coverage shows no significant correlation with any available features (r<0.15), indicating coverage is driven by
   political and institutional factors not captured in our dataset. This empirical finding validates our hybrid
  approach: ML for regions where features explain outcomes, baselines where unmeasured factors dominate.


## Revenue Model

In [141]:
df_revenue = df_active[df_active['Year'] <= 2024].copy()

In [142]:
print("Missing values in key features in coverage model:")
print(df_revenue[revenue_features].isnull().sum())

Missing values in key features in coverage model:
Type                         0
Region                       0
Income group                 0
Year                         0
Carbon_Price_USD            97
Fossil_Fuel_Dependency_%     0
Population_Log               0
dtype: int64


In [143]:
df_revenue = df_active[df_active['Year'] <= 2024].copy()                                                              
df_revenue = df_revenue.dropna(subset=['Carbon_Price_USD', 'Revenue_Million_USD', 'Fossil_Fuel_Dependency_%',
  'Population_Log'])

In [144]:
print(f"\nRevenue correlation by region:")
for region in df_revenue['Region'].unique():
    region_data = df_revenue[df_revenue['Region'] == region]
    if len(region_data) > 10:
        corr = region_data[['Population_Log', 'Carbon_Price_USD',
  'Revenue_Million_USD']].corr()['Revenue_Million_USD']['Population_Log']
        print(f"{region} (n={len(region_data)}): Population_Log corr = {corr:.3f}")


Revenue correlation by region:
Europe & Central Asia (n=338): Population_Log corr = 0.437
North America (n=132): Population_Log corr = 0.169
East Asia & Pacific (n=152): Population_Log corr = -0.272
Latin America & Caribbean (n=42): Population_Log corr = -0.068


In [145]:
X_revenue = df_revenue[revenue_features]
y_revenue = df_revenue['Revenue_Million_USD']

In [146]:
X_train_rev, X_test_rev, y_train_rev, y_test_rev = train_test_split(
      X_revenue, y_revenue, test_size=0.2, random_state=42, stratify=df_revenue['Region']
  )

In [147]:
encoders_rev={}
X_train_rev_enc = X_train_rev.copy()
X_test_rev_enc = X_test_rev.copy()


In [148]:
for feature in ['Type', 'Region', 'Income group']:
      encoders_rev[feature] = LabelEncoder()
      X_train_rev_enc[feature] = encoders_rev[feature].fit_transform(X_train_rev[feature])
      X_test_rev_enc[feature] = encoders_rev[feature].transform(X_test_rev[feature])

In [149]:
gb_rev = GradientBoostingRegressor(learning_rate=0.01, max_depth=5, min_samples_split=5, n_estimators=300,
  random_state=42)
gb_rev.fit(X_train_rev_enc, y_train_rev)
y_pred_rev = gb_rev.predict(X_test_rev_enc)

In [150]:
revenue_test_r2 = r2_score(y_test_rev, y_pred_rev)
print(f"\nREVENUE MODEL Test R²: {revenue_test_r2:.4f}")


REVENUE MODEL Test R²: 0.2611


In [151]:
print("Revenue Test R² by region:")                                                                                   
X_test_rev_results = X_test_rev.copy()  
X_test_rev_results['Predicted'] = y_pred_rev
X_test_rev_results['Actual'] = y_test_rev.values

Revenue Test R² by region:


In [152]:
for region in X_test_rev_results['Region'].unique():                                                                  
      region_data = X_test_rev_results[X_test_rev_results['Region'] == region]                                          
      if len(region_data) > 5:
          r2 = r2_score(region_data['Actual'], region_data['Predicted'])
          print(f"{region}: R² = {r2:.3f} (n={len(region_data)})")

East Asia & Pacific: R² = 0.490 (n=31)
Europe & Central Asia: R² = 0.623 (n=68)
North America: R² = -1.704 (n=26)
Latin America & Caribbean: R² = 0.554 (n=8)


In [153]:
print(f"Mean: {df_revenue['Revenue_Million_USD'].mean():.2f}")
print(f"Median: {df_revenue['Revenue_Million_USD'].median():.2f}")
print(f"Max: {df_revenue['Revenue_Million_USD'].max():.2f}")
print(f"Min: {df_revenue['Revenue_Million_USD'].min():.2f}")
print(f"Skewness: {df_revenue['Revenue_Million_USD'].skew():.2f}")

Mean: 1119.81
Median: 124.32
Max: 47330.24
Min: 0.00
Skewness: 8.59


In [154]:
print(f"\nTop 5 revenue values:")
print(df_revenue.nlargest(5, 'Revenue_Million_USD')[['Jurisdiction', 'Year', 'Revenue_Million_USD',
'Carbon_Price_USD', 'Population_Log']])


Top 5 revenue values:
     Jurisdiction  Year  Revenue_Million_USD  Carbon_Price_USD  Population_Log
2594           EU  2023          47330.23605         96.298125       19.547705
2672           EU  2024          41702.91389         61.301547       19.547321
2516           EU  2022          40407.43572         86.526108       19.542466
2438           EU  2021          36909.94455         49.779548       19.537325
2360           EU  2020          21456.28472         18.536520       19.535738


In [155]:
print(f"\nTest set - actual vs predicted (worst 5 errors):")
errors = pd.DataFrame({
      'Jurisdiction': X_test_rev['Region'].values,
      'Actual': y_test_rev.values,
      'Predicted': y_pred_rev,
      'Error': np.abs(y_test_rev.values - y_pred_rev)
  })
print(errors.nlargest(5, 'Error'))


Test set - actual vs predicted (worst 5 errors):
              Jurisdiction       Actual    Predicted        Error
65           North America     0.000000  9587.134087  9587.134087
113  Europe & Central Asia  5943.445822   224.467113  5718.978709
44   Europe & Central Asia  5315.370593  8838.556954  3523.186361
13           North America    10.574018  1500.722397  1490.148379
128    East Asia & Pacific   150.272982  1638.105773  1487.832791


Revenue is heavily right-skewed (skewness = 8.59, max = 47,330, median = 124).
Predicting log(Revenue) normalizes distribution and improves model performance

In [156]:
df_revenue['Revenue_Log'] = np.log1p(df_revenue['Revenue_Million_USD'])

In [157]:
X_revenue = df_revenue[revenue_features]
y_revenue_log = df_revenue['Revenue_Log']

In [158]:
X_train_rev, X_test_rev, y_train_rev_log, y_test_rev_log = train_test_split(
      X_revenue, y_revenue_log, test_size=0.2, random_state=42, stratify=df_revenue['Region']
  )

In [159]:
encoders_rev = {}
X_train_rev_enc = X_train_rev.copy()
X_test_rev_enc = X_test_rev.copy()

In [160]:
for feature in ['Type', 'Region', 'Income group']:
      encoders_rev[feature] = LabelEncoder()
      X_train_rev_enc[feature] = encoders_rev[feature].fit_transform(X_train_rev[feature])
      X_test_rev_enc[feature] = encoders_rev[feature].transform(X_test_rev[feature])

In [161]:
gb_rev_log = GradientBoostingRegressor(learning_rate=0.01, max_depth=5, min_samples_split=5, n_estimators=300,        
  random_state=42)
gb_rev_log.fit(X_train_rev_enc, y_train_rev_log)
y_pred_rev_log = gb_rev_log.predict(X_test_rev_enc)

In [162]:
revenue_log_r2 = r2_score(y_test_rev_log, y_pred_rev_log)
print(f"Revenue (log-transformed) Test R²: {revenue_log_r2:.4f}")

Revenue (log-transformed) Test R²: 0.6827


In [163]:
print(f"\nBy region:")
X_test_results = X_test_rev.copy()
X_test_results['Predicted_Log'] = y_pred_rev_log
X_test_results['Actual_Log'] = y_test_rev_log.values
for region in X_test_results['Region'].unique():
      region_data = X_test_results[X_test_results['Region'] == region]
      if len(region_data) > 5:
          r2 = r2_score(region_data['Actual_Log'], region_data['Predicted_Log'])
          print(f"{region}: R² = {r2:.3f} (n={len(region_data)})")


By region:
East Asia & Pacific: R² = 0.760 (n=31)
Europe & Central Asia: R² = 0.630 (n=68)
North America: R² = 0.398 (n=26)
Latin America & Caribbean: R² = 0.118 (n=8)


In [164]:
param_grid_rev = {
      'n_estimators': [200, 300, 500],
      'learning_rate': [0.01, 0.05, 0.1],
      'max_depth': [4, 5, 6],
      'min_samples_split': [5, 10, 15],
      'subsample': [0.8, 0.9, 1.0]
  }

In [165]:
gb_rev_grid = GradientBoostingRegressor(random_state=42)

In [166]:
grid_search_rev = GridSearchCV(
      gb_rev_grid,
      param_grid_rev,
      cv=5,
      scoring='r2',
      n_jobs=-1,
      verbose=1
  )

In [167]:
grid_search_rev.fit(X_train_rev_enc, y_train_rev_log)

print(f"\nBest parameters: {grid_search_rev.best_params_}")
print(f"Best CV R²: {grid_search_rev.best_score_:.4f}")

Fitting 5 folds for each of 243 candidates, totalling 1215 fits

Best parameters: {'learning_rate': 0.01, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 500, 'subsample': 0.8}
Best CV R²: 0.7861


In [168]:
best_rev_model = grid_search_rev.best_estimator_
y_pred_rev_tuned = best_rev_model.predict(X_test_rev_enc)
rev_tuned_r2 = r2_score(y_test_rev_log, y_pred_rev_tuned)

In [169]:
print(f"\nBEFORE tuning: Test R² = 0.6827")
print(f"AFTER tuning:  Test R² = {rev_tuned_r2:.4f}")
print(f"Improvement: {rev_tuned_r2 - 0.6827:.4f}")


BEFORE tuning: Test R² = 0.6827
AFTER tuning:  Test R² = 0.7177
Improvement: 0.0350


In [170]:
print("Revenue Model (Tuned) - Test R² by region:")
X_test_rev_tuned = X_test_rev.copy()                                                                                  
X_test_rev_tuned['Predicted_Log'] = y_pred_rev_tuned
X_test_rev_tuned['Actual_Log'] = y_test_rev_log.values
for region in X_test_rev_tuned['Region'].unique():
      region_data = X_test_rev_tuned[X_test_rev_tuned['Region'] == region]
      if len(region_data) > 5:
          r2 = r2_score(region_data['Actual_Log'], region_data['Predicted_Log'])
          print(f"{region}: R² = {r2:.3f} (n={len(region_data)})")

Revenue Model (Tuned) - Test R² by region:
East Asia & Pacific: R² = 0.784 (n=31)
Europe & Central Asia: R² = 0.678 (n=68)
North America: R² = 0.359 (n=26)
Latin America & Caribbean: R² = 0.531 (n=8)


In [171]:
print(f"\nOverall Test R²: {rev_tuned_r2:.4f}")
print(f"Best hyperparameters: {grid_search_rev.best_params_}")


Overall Test R²: 0.7177
Best hyperparameters: {'learning_rate': 0.01, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 500, 'subsample': 0.8}


##### Revenue Model Deep Diagnostic - Finding All Possible Improvement                                                                                                          Checking:
  1. Feature correlations and missing features (GDP?)                                                                   
  2. Interaction terms that make economic sense
  3. Outliers and data quality issues
  4. Alternative algorithms

In [172]:
print("\nCorrelation with log(Revenue):")
correlation_features = ['Year', 'Carbon_Price_USD', 'Fossil_Fuel_Dependency_%',
                          'Population_Log', 'Annual_CO2_emissions', 'GDP', 'Revenue_Log']
available_features = [f for f in correlation_features if f in df_revenue.columns]
corr_matrix = df_revenue[available_features].corr()['Revenue_Log'].sort_values(ascending=False)
print(corr_matrix)


Correlation with log(Revenue):
Revenue_Log                 1.000000
Carbon_Price_USD            0.405126
Year                        0.006245
Population_Log             -0.119682
GDP                        -0.283167
Annual_CO2_emissions       -0.395039
Fossil_Fuel_Dependency_%   -0.414703
Name: Revenue_Log, dtype: float64


In [173]:
print(f"Annual_CO2_emissions missing: {df_revenue['Annual_CO2_emissions'].isnull().sum()}")
print(f"GDP missing: {df_revenue['GDP'].isnull().sum()}")

Annual_CO2_emissions missing: 0
GDP missing: 72


In [174]:
missing_gdp = df_revenue[df_revenue['GDP'].isnull()]
print(f"Total missing GDP: {len(missing_gdp)} / {len(df_revenue)}")
print(f"\nBy jurisdiction:")
print(missing_gdp['Jurisdiction'].value_counts())

Total missing GDP: 72 / 669

By jurisdiction:
Jurisdiction
Liechtenstein                17
United Kingdom                2
Switzerland                   2
Canada                        2
Saskatchewan                  1
New Zealand                   1
Newfoundland and Labrador     1
Northwest Territories         1
Norway                        1
Nova Scotia                   1
Ontario                       1
Poland                        1
Portugal                      1
Quebec                        1
RGGI                          1
Singapore                     1
Shanghai                      1
Netherlands                   1
Slovenia                      1
South Africa                  1
Spain                         1
Sweden                        1
Tamaulipas                    1
Ukraine                       1
Uruguay                       1
Washington                    1
New Brunswick                 1
Montenegro                    1
Albania                       1
Estonia      

In [175]:
missing_gdp = df_revenue[df_revenue['GDP'].isnull()]
print(missing_gdp['Year'].value_counts().sort_index())

Year
2008     1
2009     1
2010     1
2011     1
2012     1
2013     1
2014     1
2015     1
2016     1
2017     1
2018     1
2019     1
2020     1
2021     1
2022     1
2023     1
2024    56
Name: count, dtype: int64


In [176]:
missing_gdp = df_revenue[df_revenue['GDP'].isnull()]
for year in range(2008, 2024):                                                                                             
    year_missing = missing_gdp[missing_gdp['Year'] == year]
    if len(year_missing) == 1:                                                                                        
        print(f"{year}: {year_missing['Jurisdiction'].values[0]}")

2008: Liechtenstein
2009: Liechtenstein
2010: Liechtenstein
2011: Liechtenstein
2012: Liechtenstein
2013: Liechtenstein
2014: Liechtenstein
2015: Liechtenstein
2016: Liechtenstein
2017: Liechtenstein
2018: Liechtenstein
2019: Liechtenstein
2020: Liechtenstein
2021: Liechtenstein
2022: Liechtenstein
2023: Liechtenstein


In [177]:
revenue_features_v2 = ['Type', 'Region', 'Income group', 'Year', 'Carbon_Price_USD',
                         'Fossil_Fuel_Dependency_%', 'Population_Log', 'Annual_CO2_emissions']

In [178]:
X_revenue_v2 = df_revenue[revenue_features_v2]
y_revenue_log = df_revenue['Revenue_Log']

In [179]:
X_train_v2, X_test_v2, y_train_log_v2, y_test_log_v2 = train_test_split(
      X_revenue_v2, y_revenue_log, test_size=0.2, random_state=42, stratify=df_revenue['Region']
  )

In [180]:
encoders_v2 = {}
X_train_v2_enc = X_train_v2.copy()
X_test_v2_enc = X_test_v2.copy()

In [181]:
for feature in ['Type', 'Region', 'Income group']:
      encoders_v2[feature] = LabelEncoder()
      X_train_v2_enc[feature] = encoders_v2[feature].fit_transform(X_train_v2[feature])
      X_test_v2_enc[feature] = encoders_v2[feature].transform(X_test_v2[feature])

In [182]:
gb_v2 = GradientBoostingRegressor(learning_rate=0.01, max_depth=5, min_samples_split=5,
                                     n_estimators=500, subsample=0.8, random_state=42)
gb_v2.fit(X_train_v2_enc, y_train_log_v2)
y_pred_v2 = gb_v2.predict(X_test_v2_enc)

In [183]:
r2_v2 = r2_score(y_test_log_v2, y_pred_v2)


In [184]:
print(f"WITHOUT CO2 (7 features): R² = 0.7177")
print(f"WITH CO2 (8 features):    R² = {r2_v2:.4f}")
print(f"Improvement: {r2_v2 - 0.7177:.4f}")

WITHOUT CO2 (7 features): R² = 0.7177
WITH CO2 (8 features):    R² = 0.7071
Improvement: -0.0106


In [185]:
df_revenue_final = df_active[df_active['Year'] <= 2024].copy()

In [186]:
liechtenstein_gdp = {
      2008: 5.08e9, 2009: 4.50e9, 2010: 5.08e9, 2011: 5.74e9,
      2012: 5.46e9, 2013: 6.39e9, 2014: 6.66e9, 2015: 6.27e9,
      2016: 6.24e9, 2017: 6.47e9, 2018: 6.69e9, 2019: 6.44e9,
      2020: 6.41e9, 2021: 7.91e9, 2022: 7.38e9, 2023: 8.29e9,
      2024: 8.30e9
  }

In [187]:
for year, gdp in liechtenstein_gdp.items():
      mask = (df_revenue_final['Jurisdiction'] == 'Liechtenstein') & (df_revenue_final['Year'] == year)
      df_revenue_final.loc[mask, 'GDP'] = gdp

jurisdictions_2024 = ['Albania', 'Montenegro', 'Poland', 'Portugal']
for jurisdiction in jurisdictions_2024:
      gdp_2023 = df_revenue_final[(df_revenue_final['Jurisdiction'] == jurisdiction) & (df_revenue_final['Year'] ==     
  2023)]['GDP'].values
      if len(gdp_2023) > 0:
          mask = (df_revenue_final['Jurisdiction'] == jurisdiction) & (df_revenue_final['Year'] == 2024)
          df_revenue_final.loc[mask, 'GDP'] = gdp_2023[0]

In [188]:
df_revenue_final = df_revenue_final.dropna(subset=['Carbon_Price_USD', 'Revenue_Million_USD',
  'Fossil_Fuel_Dependency_%', 'Population_Log', 'GDP'])
df_revenue_final['Revenue_Log'] = np.log1p(df_revenue_final['Revenue_Million_USD'])

In [189]:
print(f"Final revenue dataset: {len(df_revenue_final)} rows")
print(f"Missing values: {df_revenue_final[['Carbon_Price_USD', 'GDP', 'Revenue_Log']].isnull().sum()}")

Final revenue dataset: 618 rows
Missing values: Carbon_Price_USD    0
GDP                 0
Revenue_Log         0
dtype: int64


In [190]:
revenue_features_final = ['Type', 'Region', 'Income group', 'Year', 'Carbon_Price_USD',
                            'Fossil_Fuel_Dependency_%', 'Population_Log', 'GDP']

In [191]:
X_rev_final = df_revenue_final[revenue_features_final]
y_rev_final = df_revenue_final['Revenue_Log']

In [192]:
X_train_rev_f, X_test_rev_f, y_train_rev_f, y_test_rev_f = train_test_split(
      X_rev_final, y_rev_final, test_size=0.2, random_state=42, stratify=df_revenue_final['Region']
  )

In [193]:
encoders_final = {}
X_train_enc_f = X_train_rev_f.copy()
X_test_enc_f = X_test_rev_f.copy()

In [194]:
for feature in ['Type', 'Region', 'Income group']:
      encoders_final[feature] = LabelEncoder()
      X_train_enc_f[feature] = encoders_final[feature].fit_transform(X_train_rev_f[feature])
      X_test_enc_f[feature] = encoders_final[feature].transform(X_test_rev_f[feature])

In [195]:
gb_final = GradientBoostingRegressor(learning_rate=0.01, max_depth=5, min_samples_split=5,
                                       n_estimators=500, subsample=0.8, random_state=42)
gb_final.fit(X_train_enc_f, y_train_rev_f)
y_pred_final = gb_final.predict(X_test_enc_f)

In [196]:
r2_final = r2_score(y_test_rev_f, y_pred_final)

In [197]:
print(f"\nWITHOUT GDP (7 features, 669 rows): R² = 0.7177")
print(f"WITH GDP (8 features, {len(df_revenue_final)} rows): R² = {r2_final:.4f}")
print(f"Improvement: {r2_final - 0.7177:.4f}")


WITHOUT GDP (7 features, 669 rows): R² = 0.7177
WITH GDP (8 features, 618 rows): R² = 0.8683
Improvement: 0.1506


In [198]:
print("Revenue Model (with GDP) - Test R² by region:")                                                                
X_test_final_results = X_test_rev_f.copy()
X_test_final_results['Predicted_Log'] = y_pred_final
X_test_final_results['Actual_Log'] = y_test_rev_f.values
for region in X_test_final_results['Region'].unique():
      region_data = X_test_final_results[X_test_final_results['Region'] == region]
      if len(region_data) > 5:
          r2 = r2_score(region_data['Actual_Log'], region_data['Predicted_Log'])
          print(f"{region}: R² = {r2:.3f} (n={len(region_data)})")



Revenue Model (with GDP) - Test R² by region:
Europe & Central Asia: R² = 0.921 (n=64)
North America: R² = 0.462 (n=23)
East Asia & Pacific: R² = 0.864 (n=29)
Latin America & Caribbean: R² = 0.643 (n=7)


In [199]:
df_revenue_test = df_revenue_final.copy()
df_revenue_test['GDP_Log'] = np.log1p(df_revenue_test['GDP'])

In [200]:
revenue_features_log_gdp = ['Type', 'Region', 'Income group', 'Year', 'Carbon_Price_USD',                             
                              'Fossil_Fuel_Dependency_%', 'Population_Log', 'GDP_Log']

In [201]:
X_test1 = df_revenue_test[revenue_features_log_gdp]
y_test1 = df_revenue_test['Revenue_Log']

In [202]:
X_train_t1, X_test_t1, y_train_t1, y_test_t1 = train_test_split(
    X_test1, y_test1, test_size=0.2, random_state=42, stratify=df_revenue_test['Region']
  )

In [203]:
encoders_t1 = {}
X_train_t1_enc = X_train_t1.copy()
X_test_t1_enc = X_test_t1.copy()

In [204]:
for feature in ['Type', 'Region', 'Income group']:
      encoders_t1[feature] = LabelEncoder()
      X_train_t1_enc[feature] = encoders_t1[feature].fit_transform(X_train_t1[feature])
      X_test_t1_enc[feature] = encoders_t1[feature].transform(X_test_t1[feature])

In [205]:
gb_t1 = GradientBoostingRegressor(learning_rate=0.01, max_depth=5, min_samples_split=5,
                                     n_estimators=500, subsample=0.8, random_state=42)
gb_t1.fit(X_train_t1_enc, y_train_t1)
y_pred_t1 = gb_t1.predict(X_test_t1_enc)

In [206]:
r2_t1 = r2_score(y_test_t1, y_pred_t1)

print(f"Raw GDP: R² = 0.8683")
print(f"Log GDP: R² = {r2_t1:.4f}")
print(f"Difference: {r2_t1 - 0.8683:.4f}")

X_test_t1_results = X_test_t1.copy()
X_test_t1_results['Predicted'] = y_pred_t1
X_test_t1_results['Actual'] = y_test_t1.values
print(f"\nBy region:")
for region in ['Europe & Central Asia', 'North America', 'East Asia & Pacific', 'Latin America & Caribbean']:
    region_data = X_test_t1_results[X_test_t1_results['Region'] == region]
    if len(region_data) > 5:
        r2 = r2_score(region_data['Actual'], region_data['Predicted'])
        print(f"{region}: R² = {r2:.3f}")

Raw GDP: R² = 0.8683
Log GDP: R² = 0.8705
Difference: 0.0022

By region:
Europe & Central Asia: R² = 0.924
North America: R² = 0.466
East Asia & Pacific: R² = 0.865
Latin America & Caribbean: R² = 0.634


In [207]:
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(
      X_rev_final.reset_index(drop=True),                                                                                     y_rev_final.reset_index(drop=True),
      test_size=0.2,                                                                                                    
      random_state=99,
      stratify=df_revenue_final['Region'].reset_index(drop=True)
  )

In [208]:
encoders_val = {}
X_train_val_enc = X_train_val.copy()
X_test_val_enc = X_test_val.copy()

In [209]:
for feature in ['Type', 'Region', 'Income group']:                                                        
      encoders_val[feature] = LabelEncoder()
      X_train_val_enc[feature] = encoders_val[feature].fit_transform(X_train_val[feature])
      X_test_val_enc[feature] = encoders_val[feature].transform(X_test_val[feature])

In [210]:
print(f"X_train data types:\n{X_train_val_enc.dtypes}")

X_train data types:
Type                          int32
Region                        int32
Income group                  int32
Year                          int64
Carbon_Price_USD            float64
Fossil_Fuel_Dependency_%    float64
Population_Log              float64
GDP                         float64
dtype: object


In [211]:
gb_val = GradientBoostingRegressor(learning_rate=0.01, max_depth=5, min_samples_split=5,
                                      n_estimators=500, subsample=0.8, random_state=42)
gb_val.fit(X_train_val_enc, y_train_val)
y_pred_val = gb_val.predict(X_test_val_enc)

In [212]:
r2_val = r2_score(y_test_val, y_pred_val)

print(f"random_state=42: R² = 0.8683")
print(f"random_state=99: R² = {r2_val:.4f}")

random_state=42: R² = 0.8683
random_state=99: R² = 0.8647


In [213]:
X_test_val_results = X_test_val.copy()
X_test_val_results['Predicted'] = y_pred_val
X_test_val_results['Actual'] = y_test_val.values

print(f"\nBy region (random_state=99):")
for region in ['Europe & Central Asia', 'North America', 'East Asia & Pacific', 'Latin America & Caribbean']:
    region_data = X_test_val_results[X_test_val_results['Region'] == region]
    if len(region_data) > 5:
        r2 = r2_score(region_data['Actual'], region_data['Predicted'])
        print(f"{region}: R² = {r2:.3f}")


By region (random_state=99):
Europe & Central Asia: R² = 0.844
North America: R² = 0.386
East Asia & Pacific: R² = 0.913
Latin America & Caribbean: R² = 0.874


##### **Problem Identified while testing:**

  While `gb_final` achieves excellent performance on log-transformed revenue (R² = 0.8683), we discovered a significant gap when evaluating predictions on the actual revenue scale:

  - **R² on log scale:** 0.8683 (excellent)
  - **R² on revenue scale:** 0.7396 (moderate)
  - **Median prediction error:** 31.3%
  - **Predictions with >50% error:** 34.1%

  ##### **Root Cause Analysis:**

  The log transformation optimizes for **relative errors in log space**, not **absolute errors in dollar space**. When predictions are converted back to actual revenue using `np.expm1()`, small errors in log space are exponentially amplified:

  **Example:**
  - Log prediction error: 0.98 (11% in log space) → Appears acceptable
  - Revenue prediction error: $6,298M vs $2,358M (63% in dollar space) → Unacceptable for users

  This creates a fundamental mismatch: the model is optimized for one objective (log-scale accuracy) but evaluated on another (revenue-scale accuracy).

  ##### **Implications for FYP:**

  Since our climate policy simulator presents revenue projections in actual dollar amounts (not logarithmic values), users care about revenue-scale accuracy. A model that performs well in log space but poorly in revenue space provides misleading information to policymakers.

### Direct Revenue Regression Method

In [214]:
revenue_features_direct = ['Type', 'Region', 'Income group', 'Year', 'Carbon_Price_USD',
                             'Fossil_Fuel_Dependency_%', 'Population_Log', 'GDP']

In [215]:
df_revenue_direct = df_active[df_active['Year'] <= 2024].copy() 

In [216]:
df_revenue_direct = df_revenue_direct.dropna(subset=['Carbon_Price_USD', 'Revenue_Million_USD',
                                                        'Fossil_Fuel_Dependency_%', 'Population_Log', 'GDP'])

In [217]:
X_direct = df_revenue_direct[revenue_features_direct]
y_direct = df_revenue_direct['Revenue_Million_USD'] 

In [218]:
print(f"  Min: ${y_direct.min():,.2f}M")
print(f"  Max: ${y_direct.max():,.2f}M")
print(f"  Mean: ${y_direct.mean():,.2f}M")
print(f"  Median: ${y_direct.median():,.2f}M")

  Min: $0.00M
  Max: $47,330.24M
  Mean: $1,083.95M
  Median: $139.00M


In [219]:
X_train_dir, X_test_dir, y_train_dir, y_test_dir = train_test_split(
      X_direct, y_direct,
      test_size=0.2,
      random_state=42  
  )

In [220]:
encoders_direct = {}
X_train_dir_enc = X_train_dir.copy()
X_test_dir_enc = X_test_dir.copy()

In [221]:
for feature in ['Type', 'Region', 'Income group']:
      encoders_direct[feature] = LabelEncoder()
      X_train_dir_enc[feature] = encoders_direct[feature].fit_transform(X_train_dir[feature])
      X_test_dir_enc[feature] = encoders_direct[feature].transform(X_test_dir[feature])

In [222]:
gb_direct = GradientBoostingRegressor(
      loss='huber',        
      alpha=0.9,           
      learning_rate=0.01,
      max_depth=5,
      min_samples_split=5,
      n_estimators=500,
      subsample=0.8,
      random_state=42
  )


In [223]:
gb_direct.fit(X_train_dir_enc, y_train_dir)

GradientBoostingRegressor(learning_rate=0.01, loss='huber', max_depth=5,
                          min_samples_split=5, n_estimators=500,
                          random_state=42, subsample=0.8)

In [224]:
y_pred_dir_train = gb_direct.predict(X_train_dir_enc)
y_pred_dir_test = gb_direct.predict(X_test_dir_enc)

In [225]:
y_pred_dir_train = np.maximum(y_pred_dir_train, 0)
y_pred_dir_test = np.maximum(y_pred_dir_test, 0)

In [226]:
r2_train = r2_score(y_train_dir, y_pred_dir_train)

In [227]:
r2_test = r2_score(y_test_dir, y_pred_dir_test)
mae_test = mean_absolute_error(y_test_dir, y_pred_dir_test)

In [228]:
print(f"  Training set: {r2_train:.4f}")
print(f"  Test set:     {r2_test:.4f}")
print(f"\nMAE: ${mae_test:,.2f}M")

  Training set: 0.9353
  Test set:     0.8676

MAE: $547.40M


In [229]:
non_zero_mask = y_test_dir >= 10

In [230]:
if non_zero_mask.sum() > 0:
      mape_test = mean_absolute_percentage_error(y_test_dir[non_zero_mask], y_pred_dir_test[non_zero_mask])
      errors_pct = abs(y_test_dir[non_zero_mask] - y_pred_dir_test[non_zero_mask]) / y_test_dir[non_zero_mask] * 100    

      print(f"\nPercentage Errors (revenues >= $10M):")
      print(f"  Sample size: {non_zero_mask.sum()} predictions")
      print(f"  MAPE: {mape_test*100:.2f}%")
      print(f"  Median error: {errors_pct.median():.1f}%")
      print(f"  Mean error: {errors_pct.mean():.1f}%")
      print(f"  25th percentile: {errors_pct.quantile(0.25):.1f}%")
      print(f"  75th percentile: {errors_pct.quantile(0.75):.1f}%")

      print(f"\n📊 Error Distribution:")
      print(f"  < 10% error:  {sum(errors_pct < 10)} predictions ({sum(errors_pct < 10)/len(errors_pct)*100:.1f}%)")    
      print(f"  10-20% error: {sum((errors_pct >= 10) & (errors_pct < 20))} predictions ({sum((errors_pct >= 10) & (errors_pct < 20))/len(errors_pct)*100:.1f}%)")
      print(f"  20-30% error: {sum((errors_pct >= 20) & (errors_pct < 30))} predictions ({sum((errors_pct >= 20) & (errors_pct < 30))/len(errors_pct)*100:.1f}%)")
      print(f"  30-50% error: {sum((errors_pct >= 30) & (errors_pct < 50))} predictions ({sum((errors_pct >= 30) & (errors_pct < 50))/len(errors_pct)*100:.1f}%)")
      print(f"  > 50% error:  {sum(errors_pct >= 50)} predictions ({sum(errors_pct >= 50)/len(errors_pct)*100:.1f}%) ")


Percentage Errors (revenues >= $10M):
  Sample size: 78 predictions
  MAPE: 218.73%
  Median error: 26.8%
  Mean error: 218.7%
  25th percentile: 11.8%
  75th percentile: 82.0%

📊 Error Distribution:
  < 10% error:  19 predictions (24.4%)
  10-20% error: 11 predictions (14.1%)
  20-30% error: 13 predictions (16.7%)
  30-50% error: 9 predictions (11.5%)
  > 50% error:  26 predictions (33.3%) 


In [231]:
comparison_df = pd.DataFrame({
      'Actual_Revenue': y_test_dir[:10].values,
      'Predicted_Revenue': y_pred_dir_test[:10],
      'Error_$M': abs(y_test_dir[:10].values - y_pred_dir_test[:10]),
      'Error_%': abs(y_test_dir[:10].values - y_pred_dir_test[:10]) / np.maximum(y_test_dir[:10].values, 1) * 100
  })
print(comparison_df.to_string(index=False))

 Actual_Revenue  Predicted_Revenue   Error_$M      Error_%
     174.636989         218.786027  44.149039    25.280463
     313.929632          83.049296 230.880336    73.545251
       0.000000         625.549266 625.549266 62554.926613
    1080.118042        1297.981970 217.863928    20.170381
       4.767840          21.955223  17.187383   360.485740
    1149.135735         214.997545 934.138190    81.290500
       0.000000          22.844711  22.844711  2284.471103
       0.000000         913.618468 913.618468 91361.846797
     349.356435         476.831108 127.474673    36.488429
    1237.135167        1203.707010  33.428157     2.702062


In [232]:
print("COMPARISON: Direct vs Log-Based Model")

print(f"\nLog-based model (gb_final):")
print(f"  R² on log scale: 0.8683")
print(f"  R² on revenue scale: 0.7396")
print(f"  Median error: 31.3%")
print(f"  Predictions >50% error: 34.1%")

print(f"\nDirect model (gb_direct):")
print(f"  R² on revenue scale: {r2_test:.4f}")

COMPARISON: Direct vs Log-Based Model

Log-based model (gb_final):
  R² on log scale: 0.8683
  R² on revenue scale: 0.7396
  Median error: 31.3%
  Predictions >50% error: 34.1%

Direct model (gb_direct):
  R² on revenue scale: 0.8676


In [233]:
if non_zero_mask.sum() > 0:
      print(f"  Median error: {errors_pct.median():.1f}%")
      print(f"  Predictions >50% error: {sum(errors_pct >= 50)/len(errors_pct)*100:.1f}%")

  Median error: 26.8%
  Predictions >50% error: 33.3%


##### **Alternative Model: Direct Revenue Regression with Huber Loss**

  ##### **Methodology:**

  Instead of predicting log-transformed revenue, we train a model to predict actual revenue values directly:

  **Key Differences from Log-Based Model:**

  1. **Target Variable:**
     - Previous: `Revenue_Log = log1p(Revenue_Million_USD)`
     - New: `Revenue_Million_USD` (direct, no transformation)

  2. **Loss Function:**
     - Previous: Squared error on log scale
     - New: **Huber loss** (robust to outliers)

  3. **Optimization Objective:**
     - Previous: Minimize errors in log space
     - New: Minimize errors in revenue space (what users see)

  ##### **Why Huber Loss?**

  Revenue data has extreme outliers ($1M to $47,330M range). Standard squared loss would be dominated by large revenue predictions. Huber loss provides robustness:

  - For small errors: Acts like squared loss (smooth gradient)
  - For large errors: Acts like absolute loss (prevents outlier dominance)
  - Threshold (α = 0.9): Balances sensitivity and robustness

  **Mathematical Property:**
  L_δ(r) = { ½r²           for |r| ≤ δ
           { δ(|r| - ½δ)   otherwise

  This allows the model to learn from the full revenue range without being overly influenced by extreme values like the EU ETS ($47B).

  ##### **Model Configuration:**

  ```python
  GradientBoostingRegressor(
      loss='huber',        # Robust loss function
      alpha=0.9,           # Huber threshold
      learning_rate=0.01,  # Same as log-based model
      max_depth=5,         # Same tree depth
      min_samples_split=5, # Same splitting criterion
      n_estimators=500,    # Same ensemble size
      subsample=0.8,       # Same bagging ratio
      random_state=42      # Same random seed for reproducibility
  )

  We maintain identical hyperparameters to gb_final (except loss function) for fair comparison.

  Expected Trade-offs:

  Advantages:
  - Direct optimization for revenue-scale accuracy
  - No exponential amplification of errors
  - Predictions directly interpretable in dollars

  Potential Disadvantages:
  - May struggle with extreme outliers more than log model
  - Could have higher variance on large revenue predictions
  - Might not handle zero revenues as gracefully

  Evaluation Criteria:

  We compared both models on revenue-scale metrics:
  - R² on actual revenue (both models evaluated consistently)
  - Median percentage error (robust to outliers)
  - Distribution of errors (<10%, 10-20%, 20-30%, 30-50%, >50%)
  - Performance by revenue magnitude (small vs large policies)



## Success Model (Policy Survival Prediction)

  **Goal:** Predict whether a carbon pricing policy will survive or be abolished

  **Model Type:** Binary Classification (Logistic Regression, Random Forest, Gradient Boosting)

  **Target Variable:** Status (Implemented vs Abolished)

  **Use Case:** Assess political feasibility and survival probability of proposed policies

### Data Exploration for Success Model

In [234]:
print("1. STATUS DISTRIBUTION (Full Dataset):")
print(df['Status'].value_counts())
print(f"Total rows: {len(df)}")
status_counts = df['Status'].value_counts()
for status in status_counts.index:
      pct = status_counts[status] / len(df) * 100
      print(f"  {status}: {status_counts[status]} ({pct:.1f}%)")


1. STATUS DISTRIBUTION (Full Dataset):
Status
Implemented    2412
Abolished       396
Name: count, dtype: int64
Total rows: 2808
  Implemented: 2412 (85.9%)
  Abolished: 396 (14.1%)


In [235]:
imbalance_ratio = status_counts.iloc[0] / status_counts.iloc[1]
print(f"\nClass imbalance ratio: {imbalance_ratio:.2f}:1")
if imbalance_ratio > 3:
    print("Imbalanced dataset - will use class weights")
else:
    print("Dataset is reasonably balanced")


Class imbalance ratio: 6.09:1
Imbalanced dataset - will use class weights


Status by Region

In [236]:
region_status = pd.crosstab(df['Region'], df['Status'], margins=True)
region_pct = pd.crosstab(df['Region'], df['Status'], normalize='index') * 100

In [237]:
print("\nAbsolute counts:")
print(region_status)


Absolute counts:
Status                      Abolished  Implemented   All
Region                                                  
East Asia & Pacific                36          612   648
Europe & Central Asia               0          972   972
Latin America & Caribbean          36          288   324
Middle East & North Africa          0           36    36
North America                     324          468   792
Sub-Saharan Africa                  0           36    36
All                               396         2412  2808


In [238]:
print("Percentages (by region):")
print(region_pct.round(1))

Percentages (by region):
Status                      Abolished  Implemented
Region                                            
East Asia & Pacific               5.6         94.4
Europe & Central Asia             0.0        100.0
Latin America & Caribbean        11.1         88.9
Middle East & North Africa        0.0        100.0
North America                    40.9         59.1
Sub-Saharan Africa                0.0        100.0


In [239]:
print("Abolishment rate by region:")
for region in df['Region'].unique():
      region_data = df[df['Region'] == region]
      abolished = (region_data['Status'] == 'Abolished').sum()
      total = len(region_data)
      rate = abolished / total * 100
      print(f"  {region}: {abolished}/{total} = {rate:.1f}% abolished")

Abolishment rate by region:
  Europe & Central Asia: 0/972 = 0.0% abolished
  North America: 324/792 = 40.9% abolished
  Latin America & Caribbean: 36/324 = 11.1% abolished
  East Asia & Pacific: 36/648 = 5.6% abolished
  Middle East & North Africa: 0/36 = 0.0% abolished
  Sub-Saharan Africa: 0/36 = 0.0% abolished


In [240]:
type_status = pd.crosstab(df['Type'], df['Status'], margins=True)
type_pct = pd.crosstab(df['Type'], df['Status'], normalize='index') * 100

In [241]:
print("\nAbsolute counts:")
print(type_status)


Absolute counts:
Status      Abolished  Implemented   All
Type                                    
Carbon tax        288         1116  1404
ETS               108         1296  1404
All               396         2412  2808


In [242]:
print("Percentages (by type):")
print(type_pct.round(1))

Percentages (by type):
Status      Abolished  Implemented
Type                              
Carbon tax       20.5         79.5
ETS               7.7         92.3


In [243]:
print("Abolishment rate by type:")
for policy_type in df['Type'].unique():
      type_data = df[df['Type'] == policy_type]
      abolished = (type_data['Status'] == 'Abolished').sum()
      total = len(type_data)
      rate = abolished / total * 100
      print(f"  {policy_type}: {abolished}/{total} = {rate:.1f}% abolished")

Abolishment rate by type:
  Carbon tax: 288/1404 = 20.5% abolished
  ETS: 108/1404 = 7.7% abolished


In [244]:
income_status = pd.crosstab(df['Income group'], df['Status'], margins=True)
income_pct = pd.crosstab(df['Income group'], df['Status'], normalize='index') * 100

In [245]:
print("Absolute counts:")
print(income_status)

print("\nPercentages (by income group):")
print(income_pct.round(1))

Absolute counts:
Status               Abolished  Implemented   All
Income group                                     
High income                360         1656  2016
Upper middle income         36          756   792
All                        396         2412  2808

Percentages (by income group):
Status               Abolished  Implemented
Income group                               
High income               17.9         82.1
Upper middle income        4.5         95.5


##### Key Findings from Success Model Exploration

  **Discovery: Strong Regional Heterogeneity in Policy Survival**

  Our analysis reveals that policy survival is NOT random - it follows clear patterns that make machine learning viable:

  ##### 1. Regional Political Stability (Strongest Signal)
  - **North America: 40.9% abolished** (324/792 policy-years)
    - Political instability: Carbon pricing policies frequently reversed
    - Alberta carbon tax abolished, then replaced with TIER (ETS)
    - U.S. subnational policies vulnerable to political changes

  - **Europe: 0% abolished** (0/972 policy-years)
    - Perfect survival rate across all European policies
    - Strong institutional commitment to climate action
    - EU-level coordination provides political stability

  - **Latin America: 11.1% abolished** (36/324)
  - **East Asia: 5.6% abolished** (36/648)
  - **Middle East & Africa: 0% abolished** (72 policy-years combined, limited data)

  **Insight:** Region alone explains 80%+ of policy survival variation. This is the most important predictor.

  ##### 2. Policy Type Matters (Moderate Signal)
  - **Carbon Tax: 20.5% abolished** (288/1,404)
    - Direct taxation = more visible to voters = higher political risk
    - Alberta Carbon Tax, Australian Carbon Tax both abolished

  - **ETS: 7.7% abolished** (108/1,404)
    - Cap-and-trade systems are 2.7x more stable
    - Less visible to consumers (industry-focused)
    - More flexible design allows political adjustments without full abolishment

  **Insight:** Policy design affects political durability. ETS systems have structural advantages for survival.

  ##### 3. Income Level (Weaker Signal)
  - **High income: 17.9% abolished** (360/2,016)
    - More policy experimentation = more failures
    - Strong democratic accountability can lead to reversals

  - **Upper-middle income: 4.5% abolished** (36/792)
    - Fewer policies, more careful implementation
    - Less political volatility once implemented

  ##### 4. Class Imbalance: 6.09:1 (Manageable)
  - 85.9% Implemented vs 14.1% Abolished
  - **Strategy:** Use class weights in model training
  - **Sufficient minority class:** 396 abolished cases is enough to learn failure patterns

In [246]:
success_features = [
      'Type',                      
      'Region',                    
      'Income group',             
      'Year',                      
      'Carbon_Price_USD',          
      'Fossil_Fuel_Dependency_%',  
      'GDP'                        
  ]

In [247]:
df_success = df.copy()

In [248]:
print("Missing Values Check:")
for feature in success_features:
    missing = df_success[feature].isnull().sum()
    pct = missing / len(df_success) * 100
    print(f"  {feature}: {missing} ({pct:.1f}%)")

Missing Values Check:
  Type: 0 (0.0%)
  Region: 0 (0.0%)
  Income group: 0 (0.0%)
  Year: 0 (0.0%)
  Carbon_Price_USD: 1988 (70.8%)
  Fossil_Fuel_Dependency_%: 78 (2.8%)
  GDP: 190 (6.8%)


In [249]:
print(f"Target variable (Status) missing: {df_success['Status'].isnull().sum()}")

Target variable (Status) missing: 0


In [250]:
print("Handling Missing Values:")
gdp_missing_before = df_success['GDP'].isnull().sum()
print(f"  GDP missing before: {gdp_missing_before}")


Handling Missing Values:
  GDP missing before: 190


In [251]:
liechtenstein_gdp = {
      2008: 5.08e9, 2009: 4.50e9, 2010: 5.08e9, 2011: 5.74e9,
      2012: 5.46e9, 2013: 6.39e9, 2014: 6.66e9, 2015: 6.27e9,
      2016: 6.24e9, 2017: 6.47e9, 2018: 6.69e9, 2019: 6.44e9,
      2020: 6.41e9, 2021: 7.91e9, 2022: 7.38e9, 2023: 8.29e9
  }

In [252]:
for year, gdp in liechtenstein_gdp.items():
      mask = (df_success['Jurisdiction'] == 'Liechtenstein') & (df_success['Year'] == year)
      df_success.loc[mask, 'GDP'] = gdp

In [253]:
jurisdictions_2024 = df_success[df_success['Year'] == 2024]['Jurisdiction'].unique()
for jurisdiction in jurisdictions_2024:
    gdp_2023 = df_success[(df_success['Jurisdiction'] == jurisdiction) &
                            (df_success['Year'] == 2023)]['GDP'].values
    if len(gdp_2023) > 0 and not pd.isna(gdp_2023[0]):
        mask = (df_success['Jurisdiction'] == jurisdiction) & (df_success['Year'] == 2024)
        df_success.loc[mask, 'GDP'] = gdp_2023[0]
gdp_missing_after = df_success['GDP'].isnull().sum()
print(f"  GDP missing after filling: {gdp_missing_after}")

  GDP missing after filling: 96


In [254]:
price_missing = df_success['Carbon_Price_USD'].isnull().sum()
print(f"  Carbon_Price_USD missing: {price_missing} (will filter these out)")

  Carbon_Price_USD missing: 1988 (will filter these out)


In [255]:
df_success = df_success.dropna(subset=success_features + ['Status'])

In [256]:
print(f"Final Success Model dataset:")
print(f"  Total rows: {len(df_success)}")
print(f"  Features: {len(success_features)}")
print(f"  Samples per feature: {len(df_success) // len(success_features)}")

Final Success Model dataset:
  Total rows: 754
  Features: 7
  Samples per feature: 107


In [257]:
print("Final Class Distribution:")
final_status = df_success['Status'].value_counts()
print(final_status)
for status in final_status.index:
    pct = final_status[status] / len(df_success) * 100
    print(f"  {status}: {final_status[status]} ({pct:.1f}%)")

Final Class Distribution:
Status
Implemented    699
Abolished       55
Name: count, dtype: int64
  Implemented: 699 (92.7%)
  Abolished: 55 (7.3%)


In [258]:
abolished_count = (df_success['Status'] == 'Abolished').sum()
implemented_count = (df_success['Status'] == 'Implemented').sum()
print(f"Abolished cases: {abolished_count} ")
print(f"Implemented cases: {implemented_count}")

Abolished cases: 55 
Implemented cases: 699


##### Decision: Feature Selection Strategy for Success Model

  **Problem Identified:**
  After filtering for complete data with Carbon_Price_USD, we have only **55 abolished cases** (754 total rows, 7.3% minority class). This is concerning because:
  - 55 abolished cases ÷ 7 features = **7.8 samples per feature** for minority class
  - Below recommended minimum of 10-20 samples per feature
  - High risk of overfitting to limited failure examples
  - Lost 341 abolished cases (396 → 55) due to missing price data

  **Root Cause Analysis:**
  Carbon_Price_USD has 70.8% missing values (1,988 of 2,808 rows). Why?
  1. Many policies existed before systematic price reporting (pre-2010)
  2. **Abolished policies lack data** - they failed early, before comprehensive data collection
  3. Some jurisdictions don't report prices publicly
  4. Missing data is NOT random - it's correlated with policy failure

  **The Missing Data Pattern Is Informative:**
  - Policies that survived → well-documented, mature, complete price data
  - Policies that were abolished → often failed early, incomplete/missing price records
  - This creates **selection bias** if we filter by price availability

  ---

  ##### Solution: Remove Carbon_Price_USD Feature (6 Features Total)

  **Theoretical Justification:**

  **1. Political Survival ≠ Economic Parameters**
  - Policy survival is driven by **political and institutional factors**, not price levels
  - Literature on policy durability shows: institutional stability > policy design parameters
  - Examples:
    - Alberta Carbon Tax ($20-$30/ton) - Abolished despite moderate price
    - EU ETS (€20-€90/ton) - Survived with varying prices
    - **Region and Type matter more than price level**

  **2. Price Is Not The Abolishment Driver**
  - High prices don't necessarily cause abolishment (Sweden: $130/ton, survived)
  - Low prices don't guarantee survival (Australia: $23/ton, abolished)
  - Political opposition is driven by:
    - Regional political culture (North America 40.9% fail, Europe 0% fail)
    - Policy visibility (Carbon Tax 20.5% fail vs ETS 7.7% fail)
    - Not price magnitude

  **3. Price Likely Correlated With Other Features (Multicollinearity)**
  - High-income countries tend to set higher prices
  - GDP and Income group already capture economic capacity
  - Removing price doesn't lose unique information

  **4. Precedent: Same Logic As Coverage Model Decision**
  - Coverage Model: Discovered regional heterogeneity, used Europe-only ML + baselines
  - Success Model: Discovered price data limitation, removed feature to preserve sample size
  - **Both decisions prioritize data quality over feature quantity**

  ---

  ##### Revised Feature Set (6 Features):

  1. **Type** (Carbon tax vs ETS) - 20.5% vs 7.7% abolishment → **Strong signal**
  2. **Region** (World Bank regions) - 0% to 40.9% range → **STRONGEST predictor**
  3. **Income group** (High vs Upper-middle) - 17.9% vs 4.5% → **Moderate signal**
  4. **Year** (1990-2024) - Temporal trends (recent policies more stable?)
  5. **Fossil_Fuel_Dependency_%** - High fossil = stronger industry opposition?
  6. **GDP** - Economic capacity affects political stability


In [259]:
success_features = [
      'Type', 'Region', 'Income group', 'Year',
      'Fossil_Fuel_Dependency_%', 'GDP'
  ]

In [260]:
df_success = df.copy()

In [261]:
liechtenstein_gdp = {
      2008: 5.08e9, 2009: 4.50e9, 2010: 5.08e9, 2011: 5.74e9,
      2012: 5.46e9, 2013: 6.39e9, 2014: 6.66e9, 2015: 6.27e9,
      2016: 6.24e9, 2017: 6.47e9, 2018: 6.69e9, 2019: 6.44e9,
      2020: 6.41e9, 2021: 7.91e9, 2022: 7.38e9, 2023: 8.29e9
  }


In [262]:
for year, gdp in liechtenstein_gdp.items():
      mask = (df_success['Jurisdiction'] == 'Liechtenstein') & (df_success['Year'] == year)
      df_success.loc[mask, 'GDP'] = gdp

In [263]:
jurisdictions_2024 = df_success[df_success['Year'] == 2024]['Jurisdiction'].unique()
for jurisdiction in jurisdictions_2024:
      gdp_2023 = df_success[(df_success['Jurisdiction'] == jurisdiction) &
                            (df_success['Year'] == 2023)]['GDP'].values
      if len(gdp_2023) > 0 and not pd.isna(gdp_2023[0]):
          mask = (df_success['Jurisdiction'] == jurisdiction) & (df_success['Year'] == 2024)
          df_success.loc[mask, 'GDP'] = gdp_2023[0]
df_success = df_success.dropna(subset=success_features + ['Status'])


In [264]:
print(f"Total rows: {len(df_success)}")
print(f"Features: {len(success_features)}")
print(f"\nClass Distribution:")
print(df_success['Status'].value_counts())
abolished_count = (df_success['Status'] == 'Abolished').sum()
print(f"\nAbsolished cases: {abolished_count} ({abolished_count/len(df_success)*100:.1f}%)")
print(f"Samples per feature (minority class): {abolished_count // len(success_features)}")

Total rows: 2712
Features: 6

Class Distribution:
Status
Implemented    2327
Abolished       385
Name: count, dtype: int64

Absolished cases: 385 (14.2%)
Samples per feature (minority class): 64


### Train Test Split

In [265]:
X_success = df_success[success_features]
y_success = df_success['Status']

In [266]:
X_train_suc, X_test_suc, y_train_suc, y_test_suc = train_test_split(
      X_success, y_success, test_size=0.2, random_state=42, stratify=y_success
  )

In [267]:
print(f"Train set: {len(X_train_suc)} rows")
print(f"  Implemented: {(y_train_suc == 'Implemented').sum()}")
print(f"  Abolished: {(y_train_suc == 'Abolished').sum()} ({(y_train_suc == 'Abolished').sum()/len(y_train_suc)*100:.1f}%)")

print(f"\nTest set: {len(X_test_suc)} rows")
print(f"  Implemented: {(y_test_suc == 'Implemented').sum()}")
print(f"  Abolished: {(y_test_suc == 'Abolished').sum()} ({(y_test_suc == 'Abolished').sum()/len(y_test_suc)*100:.1f}%)")

Train set: 2169 rows
  Implemented: 1861
  Abolished: 308 (14.2%)

Test set: 543 rows
  Implemented: 466
  Abolished: 77 (14.2%)


In [268]:
categorical_features = ['Type', 'Region', 'Income group']
encoders_success = {}

In [269]:
X_train_suc_enc = X_train_suc.copy()
X_test_suc_enc = X_test_suc.copy()

In [270]:
for feature in categorical_features:
      encoders_success[feature] = LabelEncoder()
      X_train_suc_enc[feature] = encoders_success[feature].fit_transform(X_train_suc[feature])
      X_test_suc_enc[feature] = encoders_success[feature].transform(X_test_suc[feature])

### Model Training

In [271]:
models_success = {
      'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
      'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
      'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
  }

In [272]:
results_clean = []

In [273]:
for name, model in models_success.items():
      model.fit(X_train_suc_enc, y_train_suc)
      y_pred = model.predict(X_test_suc_enc)

      results_clean.append({
          'Model': name,
          'Accuracy': accuracy_score(y_test_suc, y_pred),
          'Precision': precision_score(y_test_suc, y_pred, pos_label='Abolished'),
          'Recall': recall_score(y_test_suc, y_pred, pos_label='Abolished'),
          'F1': f1_score(y_test_suc, y_pred, pos_label='Abolished')
      })

In [274]:
df_results_success = pd.DataFrame(results_clean).sort_values('F1', ascending=False)
print("SUCCESS MODEL PERFORMANCE:\n")
print(df_results_success.round(3).to_string(index=False))
print(f"\nBest: {df_results_success.iloc[0]['Model']} (F1={df_results_success.iloc[0]['F1']:.3f})")

SUCCESS MODEL PERFORMANCE:

              Model  Accuracy  Precision  Recall    F1
  Gradient Boosting     0.950      0.963   0.675 0.794
      Random Forest     0.827      0.430   0.675 0.525
Logistic Regression     0.858      0.000   0.000 0.000

Best: Gradient Boosting (F1=0.794)


### Hyperparameter Tuning

In [275]:
param_grid = {
      'n_estimators': [200, 300, 500],
      'learning_rate': [0.01, 0.05, 0.1],
      'max_depth': [3, 5, 7],
      'min_samples_split': [5, 10, 15]
  }

In [276]:
gb_success = GradientBoostingClassifier(random_state=42)
grid_search_success = GridSearchCV(
      gb_success, param_grid, cv=5,
      scoring='f1_weighted', n_jobs=-1, verbose=1
  )

In [277]:
grid_search_success.fit(X_train_suc_enc, y_train_suc)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'min_samples_split': [5, 10, 15],
                         'n_estimators': [200, 300, 500]},
             scoring='f1_weighted', verbose=1)

In [278]:
print("Best parameters:")
print(grid_search_success.best_params_)

Best parameters:
{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_split': 5, 'n_estimators': 200}


In [279]:
gb_success_final = grid_search_success.best_estimator_
y_pred_final = gb_success_final.predict(X_test_suc_enc)
print(f"\nFINAL TUNED MODEL PERFORMANCE:")
print(f"Accuracy:  {accuracy_score(y_test_suc, y_pred_final):.3f}")
print(f"Precision: {precision_score(y_test_suc, y_pred_final, pos_label='Abolished'):.3f}")
print(f"Recall:    {recall_score(y_test_suc, y_pred_final, pos_label='Abolished'):.3f}")
print(f"F1-Score:  {f1_score(y_test_suc, y_pred_final, pos_label='Abolished'):.3f}")


FINAL TUNED MODEL PERFORMANCE:
Accuracy:  0.954
Precision: 1.000
Recall:    0.675
F1-Score:  0.806


In [280]:
y_test_proba_abolished = gb_success_final.predict_proba(X_test_suc_enc)[:, 0]

In [281]:
LOW_RISK_THRESHOLD = 0.35
HIGH_RISK_THRESHOLD = 0.65

In [282]:
def categorize_risk(probability):
      """
      Convert abolishment probability to risk category

      Low Risk (< 0.35): Policy will likely survive
      At Risk (0.35-0.65): Uncertain outcome, requires monitoring
      High Risk (> 0.65): Policy likely to be abolished
      """
      if probability < LOW_RISK_THRESHOLD:
          return "Low Risk"
      elif probability < HIGH_RISK_THRESHOLD:
          return "At Risk"
      else:
          return "High Risk"

In [283]:
y_test_risk_categories = [categorize_risk(prob) for prob in y_test_proba_abolished]

In [284]:
binary_predictions = gb_success_final.predict(X_test_suc_enc)
print(f"\nBinary predictions (first 5): {binary_predictions[:5]}")



Binary predictions (first 5): ['Implemented' 'Implemented' 'Implemented' 'Implemented' 'Implemented']


In [285]:
print(f"\nClass labels: {gb_success_final.classes_}")


Class labels: ['Abolished' 'Implemented']


In [286]:
binary_labels = list(binary_predictions)

In [287]:
actual_labels = ['Abolished' if val == 0 else 'Implemented' for val in y_test_suc]

In [288]:
results_df = pd.DataFrame({
      'Actual_Status': actual_labels,
      'Abolishment_Probability': y_test_proba_abolished,
      'Risk_Category': y_test_risk_categories,
      'Binary_Prediction': binary_labels
  })

In [289]:
print(results_df.head(40))

   Actual_Status  Abolishment_Probability Risk_Category Binary_Prediction
0    Implemented                 0.207930      Low Risk       Implemented
1    Implemented                 0.029440      Low Risk       Implemented
2    Implemented                 0.029440      Low Risk       Implemented
3    Implemented                 0.040224      Low Risk       Implemented
4    Implemented                 0.040224      Low Risk       Implemented
5    Implemented                 0.207930      Low Risk       Implemented
6    Implemented                 0.040224      Low Risk       Implemented
7    Implemented                 0.040224      Low Risk       Implemented
8    Implemented                 0.207930      Low Risk       Implemented
9    Implemented                 0.882599     High Risk         Abolished
10   Implemented                 0.020124      Low Risk       Implemented
11   Implemented                 0.039904      Low Risk       Implemented
12   Implemented                 0.183

In [290]:
print(results_df['Risk_Category'].value_counts())
print(f"\nTotal test samples: {len(results_df)}")

Risk_Category
Low Risk     491
High Risk     52
Name: count, dtype: int64

Total test samples: 543


In [291]:
for category in ['Low Risk', 'At Risk', 'High Risk']:
      count = (results_df['Risk_Category'] == category).sum()
      percentage = count / len(results_df) * 100
      print(f"{category}: {count} ({percentage:.1f}%)")

Low Risk: 491 (90.4%)
At Risk: 0 (0.0%)
High Risk: 52 (9.6%)


In [292]:
print(f"Total test samples: {len(y_test_proba_abolished)}")
print(f"\nStatistics:")
print(f"  Min:    {y_test_proba_abolished.min():.4f}")
print(f"  Max:    {y_test_proba_abolished.max():.4f}")
print(f"  Mean:   {y_test_proba_abolished.mean():.4f}")
print(f"  Median: {np.median(y_test_proba_abolished):.4f}")
print(f"  Std:    {y_test_proba_abolished.std():.4f}")

Total test samples: 543

Statistics:
  Min:    0.0192
  Max:    0.8826
  Mean:   0.1513
  Median: 0.0402
  Std:    0.2470


In [293]:
in_at_risk_range = [(p >= 0.35) and (p <= 0.65) for p in y_test_proba_abolished]
print(f"\nProbabilities in 'At Risk' range (0.35-0.65): {sum(in_at_risk_range)}")


Probabilities in 'At Risk' range (0.35-0.65): 0


In [294]:
bins = [0, 0.1, 0.2, 0.3, 0.35, 0.5, 0.65, 0.7, 0.8, 0.9, 1.0]
for i in range(len(bins)-1):
      count = sum([(p >= bins[i]) and (p < bins[i+1]) for p in y_test_proba_abolished])
      print(f"  [{bins[i]:.2f} - {bins[i+1]:.2f}): {count} samples")


  [0.00 - 0.10): 357 samples
  [0.10 - 0.20): 60 samples
  [0.20 - 0.30): 74 samples
  [0.30 - 0.35): 0 samples
  [0.35 - 0.50): 0 samples
  [0.50 - 0.65): 0 samples
  [0.65 - 0.70): 0 samples
  [0.70 - 0.80): 0 samples
  [0.80 - 0.90): 52 samples
  [0.90 - 1.00): 0 samples


###  ADD HAS_PRIOR_POLICY FEATURE in success model 

In [295]:
countries_with_prior_policy = [
      'EU', 'European Union', 'EU ETS',
      'Germany', 'France', 'UK', 'United Kingdom', 'Sweden', 'Switzerland',
      'Norway', 'Finland', 'Denmark', 'Netherlands', 'Ireland', 'Spain',
      'Portugal', 'Austria', 'Belgium', 'Poland', 'Italy',

      'China', 'South Korea', 'Korea', 'Japan', 'New Zealand', 'Singapore',
      'Kazakhstan', 'Indonesia',

      'Canada', 'British Columbia', 'Quebec', 'Alberta', 'Nova Scotia',
      'California', 'Washington', 'Oregon', 'Massachusetts', 'New York',
      'Connecticut', 'Delaware', 'Maine', 'Maryland', 'New Hampshire',
      'Rhode Island', 'Vermont', 'RGGI',

      'Mexico', 'Chile', 'Colombia', 'Argentina',

      'South Africa'
  ]


In [296]:
df_success_v2 = df_success.copy()
df_success_v2['Has_Prior_Policy'] = df_success_v2['Jurisdiction'].isin(countries_with_prior_policy).astype(int)       


In [297]:
print(df_success_v2['Has_Prior_Policy'].value_counts())

Has_Prior_Policy
1    1470
0    1242
Name: count, dtype: int64


In [298]:
print("\nAbolishment rate by prior policy status:")
abolishment_by_prior = df_success_v2.groupby('Has_Prior_Policy')['Status'].value_counts(normalize=True).unstack()     
print(abolishment_by_prior)


Abolishment rate by prior policy status:
Status            Abolished  Implemented
Has_Prior_Policy                        
0                  0.225443     0.774557
1                  0.071429     0.928571


In [299]:
feature_cols_success_v2 = [
      'Type',
      'Region',
      'Income group',
      'Year',
      'Fossil_Fuel_Dependency_%',
      'GDP',
      'Has_Prior_Policy'  
  ]

In [300]:
X_success_v2 = df_success_v2[feature_cols_success_v2]
y_success_v2 = df_success_v2['Status']

In [301]:
X_train_suc_v2, X_test_suc_v2, y_train_suc_v2, y_test_suc_v2 = train_test_split(
      X_success_v2, y_success_v2,
      test_size=0.2,
      random_state=42,
      stratify=y_success_v2
  )

In [302]:
print(f"Train set: {len(X_train_suc_v2)} samples")
print(f"Test set: {len(X_test_suc_v2)} samples")
print(f"Train class distribution:")
print(y_train_suc_v2.value_counts())

Train set: 2169 samples
Test set: 543 samples
Train class distribution:
Status
Implemented    1861
Abolished       308
Name: count, dtype: int64


In [303]:
categorical_cols_success_v2 = ['Type', 'Region', 'Income group']

In [304]:
encoders_success_v2 = {}
X_train_suc_v2_enc = X_train_suc_v2.copy()  
X_test_suc_v2_enc = X_test_suc_v2.copy()

In [305]:
for col in categorical_cols_success_v2:
      le = LabelEncoder()
      X_train_suc_v2_enc[col] = le.fit_transform(X_train_suc_v2[col])
      X_test_suc_v2_enc[col] = le.transform(X_test_suc_v2[col])
      encoders_success_v2[col] = le

In [306]:
gb_success_v2 = GradientBoostingClassifier(
      learning_rate=0.05,
      max_depth=4,
      min_samples_split=10,
      n_estimators=300,
      subsample=0.8,
      random_state=42
  )


In [307]:
gb_success_v2.fit(X_train_suc_v2_enc, y_train_suc_v2)

GradientBoostingClassifier(learning_rate=0.05, max_depth=4,
                           min_samples_split=10, n_estimators=300,
                           random_state=42, subsample=0.8)

In [308]:
y_pred_v2 = gb_success_v2.predict(X_test_suc_v2_enc)
y_proba_v2 = gb_success_v2.predict_proba(X_test_suc_v2_enc)[:, 0]

In [309]:
accuracy_v2 = accuracy_score(y_test_suc_v2, y_pred_v2)
print (f"\nSUCCESS MODEL WITH PRIOR POLICY FEATURE PERFORMANCE:")
print(f"Accuracy:  {accuracy_v2:.3f}")


SUCCESS MODEL WITH PRIOR POLICY FEATURE PERFORMANCE:
Accuracy:  0.910


In [310]:
print(classification_report(y_test_suc_v2, y_pred_v2, digits=3))

              precision    recall  f1-score   support

   Abolished      0.675     0.701     0.688        77
 Implemented      0.950     0.944     0.947       466

    accuracy                          0.910       543
   macro avg      0.813     0.823     0.818       543
weighted avg      0.911     0.910     0.910       543



In [311]:
print(confusion_matrix(y_test_suc_v2, y_pred_v2))

[[ 54  23]
 [ 26 440]]


In [312]:
feature_importance_v2 = pd.DataFrame({
      'Feature': feature_cols_success_v2,
      'Importance': gb_success_v2.feature_importances_
  }).sort_values('Importance', ascending=False)


In [313]:
print("\nFeature Importance:")
print(feature_importance_v2.to_string(index=False))



Feature Importance:
                 Feature  Importance
                    Type    0.328254
                  Region    0.271606
                     GDP    0.149197
        Has_Prior_Policy    0.112870
Fossil_Fuel_Dependency_%    0.093965
            Income group    0.026456
                    Year    0.017651


In [314]:
test_cases_v2 = [
      {'Jurisdiction': 'Pakistan', 'Type': 'ETS', 'Region': 'Middle East & North Africa',
       'Income group': 'Upper middle income', 'Year': 2025,  # ← CORRECTED (was Lower middle income)
       'Fossil_Fuel_Dependency_%': 64.0, 'GDP': 1500.0, 'Has_Prior_Policy': 0},

      {'Jurisdiction': 'USA', 'Type': 'Carbon tax', 'Region': 'North America',
       'Income group': 'High income', 'Year': 2025,
       'Fossil_Fuel_Dependency_%': 79.8, 'GDP': 70000.0, 'Has_Prior_Policy': 1},

      {'Jurisdiction': 'Germany', 'Type': 'ETS', 'Region': 'Europe & Central Asia',
       'Income group': 'High income', 'Year': 2025,
       'Fossil_Fuel_Dependency_%': 73.5, 'GDP': 50000.0, 'Has_Prior_Policy': 1},

      {'Jurisdiction': 'China', 'Type': 'ETS', 'Region': 'East Asia & Pacific',
       'Income group': 'Upper middle income', 'Year': 2025,
       'Fossil_Fuel_Dependency_%': 84.8, 'GDP': 12500.0, 'Has_Prior_Policy': 1}
  ]

In [315]:
for test_case in test_cases_v2:
      country = test_case['Jurisdiction']

    
      input_df_v2 = pd.DataFrame([{k: v for k, v in test_case.items() if k != 'Jurisdiction'}])
      input_encoded_v2 = input_df_v2.copy()

      for col in categorical_cols_success_v2:
          input_encoded_v2[col] = encoders_success_v2[col].transform(input_df_v2[col])

     
      prediction_v2 = gb_success_v2.predict(input_encoded_v2)[0]
      proba_abolished_v2 = gb_success_v2.predict_proba(input_encoded_v2)[0, 0] * 100

      if proba_abolished_v2 < 35:
          risk_v2 = "Low Risk"
      elif proba_abolished_v2 < 65:
          risk_v2 = "At Risk"
      else:
          risk_v2 = "High Risk"

      print(f"\n{country}:")
      print(f"  Has_Prior_Policy: {test_case['Has_Prior_Policy']}")
      print(f"  Abolishment Risk (v2): {proba_abolished_v2:.1f}% [{risk_v2}]")



Pakistan:
  Has_Prior_Policy: 0
  Abolishment Risk (v2): 0.1% [Low Risk]

USA:
  Has_Prior_Policy: 1
  Abolishment Risk (v2): 17.7% [Low Risk]

Germany:
  Has_Prior_Policy: 1
  Abolishment Risk (v2): 0.0% [Low Risk]

China:
  Has_Prior_Policy: 1
  Abolishment Risk (v2): 0.0% [Low Risk]


### MODEL TESTING WITH REALISTIC SCENARIOS

In [316]:
test_europe_ets = pd.DataFrame({
      'Type': ['ETS'],
      'Region': ['Europe & Central Asia'],
      'Income group': ['High income'],
      'Year': [2024],
      'Carbon_Price_USD': [80.0],
      'Fossil_Fuel_Dependency_%': [65.0],
      'Population_Log': [np.log1p(450000000)],
      'GDP': [20000]
  })

In [317]:
test_south_asia = pd.DataFrame({
      'Type': ['Carbon tax'],
      'Region': ['South Asia'],
      'Income group': ['Lower middle income'],
      'Year': [2025],
      'Carbon_Price_USD': [10.0],
      'Fossil_Fuel_Dependency_%': [81.2],
      'Population_Log': [np.log1p(247500000)],
      'GDP': [1396]
  })

In [318]:
test_na = pd.DataFrame({
      'Type': ['Carbon tax'],
      'Region': ['North America'],
      'Income group': ['High income'],
      'Year': [2024],
      'Carbon_Price_USD': [50.0],
      'Fossil_Fuel_Dependency_%': [75.0],
      'Population_Log': [np.log1p(40000000)],
      'GDP': [2000]
  })

In [319]:
def test_revenue_prediction(test_df, case_name):
      """Test revenue model"""
      print(f"\n{case_name}:")

      
      test_encoded = test_df.copy()

 
      for feature in ['Type', 'Region', 'Income group']:
          try:
             
              value = test_df[feature].values
         
              test_encoded[feature] = encoders_val[feature].transform(value)
          except ValueError as e:
         
              original_value = test_df[feature].iloc[0]
              print(f"    WARNING: '{original_value}' not in training data for {feature}")
              print(f"      Available classes: {encoders_val[feature].classes_}")
             
              fallback = encoders_val[feature].classes_[0]
              print(f"      Using fallback: '{fallback}'")
              test_encoded[feature] = encoders_val[feature].transform([fallback])

  
      pred_log = gb_final.predict(test_encoded)[0]
      pred_revenue = np.expm1(pred_log)
      print(f"  ✓ Predicted Revenue: ${pred_revenue:,.2f} Million USD/year")
      return pred_revenue


In [320]:
def test_success_prediction(test_df, case_name):
      """Test success model"""
      print(f"\n{case_name}:")

    
      success_features = ['Type', 'Region', 'Income group', 'Year',
                         'Fossil_Fuel_Dependency_%', 'GDP']
      test_success = test_df[success_features].copy()

 
      test_encoded = test_success.copy()
      for feature in ['Type', 'Region', 'Income group']:
          try:
              value = test_success[feature].values
              test_encoded[feature] = encoders_success[feature].transform(value)
          except ValueError:
              original_value = test_success[feature].iloc[0]
              print(f"    WARNING: '{original_value}' not in training data for {feature}")
              print(f"      Available classes: {encoders_success[feature].classes_}")
              fallback = encoders_success[feature].classes_[0]
              print(f"      Using fallback: '{fallback}'")
              test_encoded[feature] = encoders_success[feature].transform([fallback])

  
      pred_proba = gb_success_final.predict_proba(test_encoded)[0]
      abolished_prob = pred_proba[0]  

      if abolished_prob < 0.35:
          risk = "Low Risk ✓"
      elif abolished_prob < 0.65:
          risk = "At Risk "
      else:
          risk = "High Risk "

      print(f"  ✓ Abolishment Probability: {abolished_prob:.3f}")
      print(f"  ✓ Risk Category: {risk}")

      return abolished_prob, risk

In [321]:
def test_success_prediction(test_df, case_name):
      """Test success model"""
      print(f"\n{case_name}:")

      
      success_cols = ['Type', 'Region', 'Income group', 'Year',
                     'Fossil_Fuel_Dependency_%', 'GDP']
      test_success = test_df[success_cols].copy()

    
      test_encoded = test_success.copy()
      for feature in ['Type', 'Region', 'Income group']:
          if feature in encoders_success:
              try:
                  test_encoded[feature] = encoders_success[feature].transform(test_success[feature])
              except ValueError:
                  print(f"  WARNING: '{test_success[feature].values[0]}' not in training")
                  fallback = encoders_success[feature].classes_[0]
                  test_encoded[feature] = encoders_success[feature].transform([fallback])

      
      pred_proba = gb_success_final.predict_proba(test_encoded)[0]
      abolished_prob = pred_proba[0]

      if abolished_prob < 0.35:
          risk = "Low Risk"
      elif abolished_prob < 0.65:
          risk = "At Risk"
      else:
          risk = "High Risk"

      print(f"  → Abolishment Probability: {abolished_prob:.3f}")
      print(f"  → Risk Category: {risk}")
      return abolished_prob, risk

In [322]:
r1 = test_revenue_prediction(test_europe_ets, "Test 1: European ETS ($80/ton)")
r2 = test_revenue_prediction(test_south_asia, "Test 2: South Asia Carbon Tax ($10/ton)")
r3 = test_revenue_prediction(test_na, "Test 3: North American Carbon Tax ($50/ton)")


Test 1: European ETS ($80/ton):
  ✓ Predicted Revenue: $12.14 Million USD/year

Test 2: South Asia Carbon Tax ($10/ton):
      Available classes: ['East Asia & Pacific' 'Europe & Central Asia' 'Latin America & Caribbean'
 'North America' 'Sub-Saharan Africa']
      Using fallback: 'East Asia & Pacific'
      Available classes: ['High income' 'Upper middle income']
      Using fallback: 'High income'
  ✓ Predicted Revenue: $15.04 Million USD/year

Test 3: North American Carbon Tax ($50/ton):
  ✓ Predicted Revenue: $3.24 Million USD/year


In [323]:
s1 = test_success_prediction(test_europe_ets, "Test 1: European ETS")
s2 = test_success_prediction(test_south_asia, "Test 2: South Asia Carbon Tax")
s3 = test_success_prediction(test_na, "Test 3: North American Carbon Tax")


Test 1: European ETS:
  → Abolishment Probability: 0.025
  → Risk Category: Low Risk

Test 2: South Asia Carbon Tax:
  → Abolishment Probability: 0.041
  → Risk Category: Low Risk

Test 3: North American Carbon Tax:
  → Abolishment Probability: 0.546
  → Risk Category: At Risk


In [324]:

print(f"\nRevenue Predictions:")
print(f"  European ETS: ${r1:,.2f}M/year")
print(f"  South Asia Tax: ${r2:,.2f}M/year")
print(f"  North America Tax: ${r3:,.2f}M/year")
print(f"\nSuccess Risk Categories:")
print(f"  European ETS: {s1[1]}")
print(f"  South Asia Tax: {s2[1]}")
print(f"  North America Tax: {s3[1]}")


Revenue Predictions:
  European ETS: $12.14M/year
  South Asia Tax: $15.04M/year
  North America Tax: $3.24M/year

Success Risk Categories:
  European ETS: Low Risk
  South Asia Tax: Low Risk
  North America Tax: At Risk


In [325]:
print("\nRevenue dataset - European ETS examples:")
eu_ets_examples = df_revenue[
      (df_revenue['Type'] == 'ETS') &
      (df_revenue['Region'] == 'Europe & Central Asia')
  ].sort_values('Revenue_Million_USD', ascending=False).head(10)
print(eu_ets_examples[['Type', 'Region', 'Income group', 'Year',
                         'Carbon_Price_USD', 'Fossil_Fuel_Dependency_%',
                         'Population_Log', 'GDP', 'Revenue_Million_USD']])


Revenue dataset - European ETS examples:
     Type                 Region Income group  Year  Carbon_Price_USD  \
2594  ETS  Europe & Central Asia  High income  2023         96.298125   
2672  ETS  Europe & Central Asia  High income  2024         61.301547   
2516  ETS  Europe & Central Asia  High income  2022         86.526108   
2438  ETS  Europe & Central Asia  High income  2021         49.779548   
2360  ETS  Europe & Central Asia  High income  2020         18.536520   
2282  ETS  Europe & Central Asia  High income  2019         24.505716   
2204  ETS  Europe & Central Asia  High income  2018         16.263720   
2676  ETS  Europe & Central Asia  High income  2024         48.370500   
2598  ETS  Europe & Central Asia  High income  2023         32.625000   
2442  ETS  Europe & Central Asia  High income  2021         29.365000   

      Fossil_Fuel_Dependency_%  Population_Log           GDP  \
2594                 69.886732       19.547705  2.301436e+13   
2672                 68.42

In [326]:
print("\n2. Let's use an ACTUAL EU example for prediction:")
if len(eu_ets_examples) > 0:
      real_eu_row = eu_ets_examples.iloc[0:1][['Type', 'Region', 'Income group', 'Year',
                                                'Carbon_Price_USD', 'Fossil_Fuel_Dependency_%',
                                                'Population_Log', 'GDP']].copy()

      print("\nReal EU ETS row features:")
      print(real_eu_row)

      actual_revenue = eu_ets_examples.iloc[0]['Revenue_Million_USD']
      print(f"\nActual Revenue: ${actual_revenue:,.2f}M")

      real_encoded = real_eu_row.copy()
      for feature in ['Type', 'Region', 'Income group']:
          real_encoded[feature] = encoders_val[feature].transform(real_eu_row[feature])

      pred_log = gb_final.predict(real_encoded)[0]
      pred_revenue = np.expm1(pred_log)

      print(f"Model Prediction: ${pred_revenue:,.2f}M")
      print(f"Error: ${abs(pred_revenue - actual_revenue):,.2f}M ({abs(pred_revenue - actual_revenue)/actual_revenue*100:.1f}%)")


2. Let's use an ACTUAL EU example for prediction:

Real EU ETS row features:
     Type                 Region Income group  Year  Carbon_Price_USD  \
2594  ETS  Europe & Central Asia  High income  2023         96.298125   

      Fossil_Fuel_Dependency_%  Population_Log           GDP  
2594                 69.886732       19.547705  2.301436e+13  

Actual Revenue: $47,330.24M
Model Prediction: $31,172.96M
Error: $16,157.27M (34.1%)


In [327]:
print("\n3. Compare YOUR test input vs REAL data:")
print("\nYour test input:")
print(f"  GDP: 20000")
print(f"  Population_Log: {np.log1p(450000000):.4f}")
print(f"  Fossil_Fuel_Dependency_%: 65.0")

print("\nReal EU data range:")
print(f"  GDP range: {df_revenue[df_revenue['Region'] == 'Europe & Central Asia']['GDP'].min():.2f} to {df_revenue[df_revenue['Region'] == 'Europe & Central Asia']['GDP'].max():.2f}")
print(f"  Population_Log range: {df_revenue[df_revenue['Region'] == 'Europe & Central Asia']['Population_Log'].min():.4f} to {df_revenue[df_revenue['Region'] == 'Europe & Central Asia']['Population_Log'].max():.4f}")
print(f"  Fossil_Fuel_% range: {df_revenue[df_revenue['Region'] == 'Europe & Central Asia']['Fossil_Fuel_Dependency_%'].min():.2f} to {df_revenue[df_revenue['Region'] == 'Europe & Central Asia']['Fossil_Fuel_Dependency_%'].max():.2f}")


3. Compare YOUR test input vs REAL data:

Your test input:
  GDP: 20000
  Population_Log: 19.9248
  Fossil_Fuel_Dependency_%: 65.0

Real EU data range:
  GDP range: 17460996093.00 to 23014355519529.00
  Population_Log range: 10.4764 to 19.5477
  Fossil_Fuel_% range: 13.92 to 99.85


In [328]:
test_sample = df_revenue.sample(10, random_state=42)

In [329]:
prediction=[]
actuals=[]

In [330]:
for idx, row in test_sample.iterrows():
     
      test_input = pd.DataFrame({
          'Type': [row['Type']],
          'Region': [row['Region']],
          'Income group': [row['Income group']],
          'Year': [row['Year']],
          'Carbon_Price_USD': [row['Carbon_Price_USD']],
          'Fossil_Fuel_Dependency_%': [row['Fossil_Fuel_Dependency_%']],
          'Population_Log': [row['Population_Log']],
          'GDP': [row['GDP']]
      })

In [331]:
test_encoded = test_input.copy()

In [332]:
for feature in ['Type', 'Region', 'Income group']:
          test_encoded[feature] = encoders_val[feature].transform(test_input[feature])

In [333]:
pred_log = gb_final.predict(test_encoded)[0]
pred_revenue = np.expm1(pred_log)
actual_revenue = row['Revenue_Million_USD']
prediction.append(pred_revenue)
actuals.append(actual_revenue)

error_pct = abs(pred_revenue - actual_revenue) / actual_revenue * 100


In [334]:
print(f"\n{row['Type']} - {row['Region'][:20]} - {row['Year']}")
print(f"  Actual: ${actual_revenue:,.2f}M")
print(f"  Predicted: ${pred_revenue:,.2f}M")
print(f"  Error: {error_pct:.1f}%")


Carbon tax - Europe & Central Asi - 2010
  Actual: $600.37M
  Predicted: $686.26M
  Error: 14.3%


In [335]:
r2 = r2_score(actuals, prediction)
mape = mean_absolute_percentage_error(actuals, prediction)
print(f"Overall R² on these 10 samples: {r2:.3f}")
print(f"Mean Absolute Percentage Error: {mape*100:.1f}%")

Overall R² on these 10 samples: nan
Mean Absolute Percentage Error: 14.3%


In [336]:
print("MODEL VERIFICATION")

print(f"\nModel being used: gb_val")
print(f"Encoders being used: encoders_val")
print(f"\nModel parameters:")
print(gb_final.get_params())




MODEL VERIFICATION

Model being used: gb_val
Encoders being used: encoders_val

Model parameters:
{'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.01, 'loss': 'squared_error', 'max_depth': 5, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 500, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 0.8, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


In [337]:
print(f"\nX_test_val shape: {X_test_val.shape}")
print(f"y_test_val shape: {y_test_val.shape}")


X_test_val shape: (124, 8)
y_test_val shape: (124,)


In [338]:
X_test_val_enc = X_test_val.copy()
for feature in ['Type', 'Region', 'Income group']:
    
    X_test_val_enc[feature] = encoders_val[feature].transform(X_test_val[feature])

In [339]:
y_pred_val_log = gb_final.predict(X_test_val_enc)
y_pred_val_revenue = np.expm1(y_pred_val_log)
y_actual_val_revenue = np.expm1(y_test_val)

In [340]:
r2 = r2_score(y_actual_val_revenue, y_pred_val_revenue)
mape = mean_absolute_percentage_error(y_actual_val_revenue, y_pred_val_revenue)
mae = mean_absolute_error(y_actual_val_revenue, y_pred_val_revenue)

In [341]:
print(f"  R² Score: {r2:.4f}")
print(f"  MAPE: {mape*100:.2f}%")
print(f"  MAE: ${mae:,.2f}M")

  R² Score: 0.9075
  MAPE: 207402582666927104.00%
  MAE: $480.98M


In [342]:
comparison_df = pd.DataFrame({
          'Actual': y_actual_val_revenue[:10],
          'Predicted': y_pred_val_revenue[:10],
          'Error_%': abs(y_actual_val_revenue[:10] - y_pred_val_revenue[:10]) / y_actual_val_revenue[:10] * 100
      })
print(comparison_df.to_string())

          Actual    Predicted     Error_%
433    32.183624    28.252580   12.214421
33   2977.555486  2670.909763   10.298573
170  3267.302400  1582.905034   51.553152
153     0.000000     2.438478         inf
242  1314.625938  1310.199077    0.336739
588    43.806647   282.551353  544.996538
22     26.288000    24.074809    8.419018
372    44.260526   113.455207  156.334974
463   311.476882   152.560351   51.020329
471    36.160113    61.369192   69.715154


In [343]:
y_pred_final_log = gb_final.predict(X_test_enc_f)
y_pred_final_revenue = np.expm1(y_pred_final_log)
y_actual_final_revenue = np.expm1(y_test_rev_f)

In [344]:
r2_final= r2_score(y_actual_final_revenue, y_pred_final_revenue)
print(f"\nFinal Model R² on FULL test set: {r2_final:.4f}")


Final Model R² on FULL test set: 0.7396


In [345]:
non_zero_mask = y_actual_final_revenue > 0
mape_final = mean_absolute_percentage_error(
      y_actual_final_revenue[non_zero_mask],
      y_pred_final_revenue[non_zero_mask]
  )

In [346]:
print(f"  MAPE (excluding zeros): {mape_final*100:.2f}%")

  MAPE (excluding zeros): 1370.44%


In [347]:
print(f"\ny_test_rev_f (first 10 values):")
print(y_test_rev_f[:10])

print(f"\nStatistics of y_test_rev_f:")
print(f"  Min: {y_test_rev_f.min():.4f}")
print(f"  Max: {y_test_rev_f.max():.4f}")
print(f"  Mean: {y_test_rev_f.mean():.4f}")


y_test_rev_f (first 10 values):
2126    8.748278
2563    7.350346
2172    7.935370
2420    6.093178
2185    6.934370
2210    0.000000
1425    6.643353
1998    7.246680
2271    7.126877
2557    1.561882
Name: Revenue_Log, dtype: float64

Statistics of y_test_rev_f:
  Min: 0.0000
  Max: 8.7483
  Mean: 4.3061


In [348]:
X_first = X_test_enc_f.iloc[0:1]
y_first_actual = y_test_rev_f.iloc[0]

pred_log = gb_final.predict(X_first)[0]

In [349]:
print(f"  y_test_rev_f value: {y_first_actual:.4f}")
print(f"  Model prediction (raw output): {pred_log:.4f}")
print(f"  After np.expm1: {np.expm1(pred_log):,.2f}")

  y_test_rev_f value: 8.7483
  Model prediction (raw output): 7.7660
  After np.expm1: 2,358.08


In [350]:
print(f"\nScenario A: y_test_rev_f is ALREADY log values")
print(f"  Actual revenue = np.expm1({y_first_actual:.4f}) = ${np.expm1(y_first_actual):,.2f}M")
print(f"  Predicted revenue = np.expm1({pred_log:.4f}) = ${np.expm1(pred_log):,.2f}M")

print(f"\nScenario B: y_test_rev_f is RAW revenue values")
print(f"  Actual revenue = ${y_first_actual:,.2f}M")
print(f"  Predicted revenue = ${pred_log:,.2f}M (if model outputs raw)")
print(f"  Predicted revenue = ${np.expm1(pred_log):,.2f}M (if model outputs log)")


Scenario A: y_test_rev_f is ALREADY log values
  Actual revenue = np.expm1(8.7483) = $6,298.83M
  Predicted revenue = np.expm1(7.7660) = $2,358.08M

Scenario B: y_test_rev_f is RAW revenue values
  Actual revenue = $8.75M
  Predicted revenue = $7.77M (if model outputs raw)
  Predicted revenue = $2,358.08M (if model outputs log)


In [351]:
print(f"\nX_train_rev_f shape: {X_train_rev_f.shape}")
print(f"y_train_rev_f shape: {y_train_rev_f.shape}")
print(f"\ny_train_rev_f first 5 values:")
print(y_train_rev_f[:5])


X_train_rev_f shape: (494, 8)
y_train_rev_f shape: (494,)

y_train_rev_f first 5 values:
1994    0.000305
2349    7.063119
1976    0.000000
1968    6.317988
2245    0.000000
Name: Revenue_Log, dtype: float64


In [352]:
if y_train_rev_f.max() < 15:
          print(f"\n✓ y_train_rev_f appears to be LOG-TRANSFORMED (max = {y_train_rev_f.max():.2f})")
else:
          print(f"\n✓ y_train_rev_f appears to be RAW REVENUE (max = {y_train_rev_f.max():,.2f})")


✓ y_train_rev_f appears to be LOG-TRANSFORMED (max = 10.76)


In [353]:
y_pred_log = gb_final.predict(X_test_enc_f)

In [354]:
y_actual_revenue = np.expm1(y_test_rev_f)
y_pred_revenue = np.expm1(y_pred_log)

In [355]:
print(f"\nConverted values (first 10):")
comparison = pd.DataFrame({
      'y_test_log': y_test_rev_f[:10].values,
      'y_pred_log': y_pred_log[:10],
      'Actual_Revenue_M': y_actual_revenue[:10],
      'Predicted_Revenue_M': y_pred_revenue[:10],
      'Error_%': abs(y_actual_revenue[:10] - y_pred_revenue[:10]) / np.maximum(y_actual_revenue[:10], 1) * 100
  })
print(comparison)


Converted values (first 10):
      y_test_log  y_pred_log  Actual_Revenue_M  Predicted_Revenue_M    Error_%
2126    8.748278    7.766028       6298.830856          2358.082547  62.563171
2563    7.350346    6.879868       1555.734888           971.497680  37.553777
2172    7.935370    7.874669       2793.392490          2628.814432   5.891691
2420    6.093178    3.538891        441.826690            33.428716  92.433975
2185    6.934370    5.993541       1025.972053           399.831271  61.029029
2210    0.000000    0.474010          0.000000             0.606423  60.642295
1425    6.643353    6.729465        766.665000           835.699815   9.004561
1998    7.246680    7.213919       1402.437476          1357.204596   3.225305
2271    7.126877    6.411570       1243.982578           607.849072  51.136850
2557    1.561882    0.774162          3.767787             1.168773  68.979862


In [356]:
y_pred_log = gb_final.predict(X_test_enc_f)
r2_log = r2_score(y_test_rev_f, y_pred_log)

print(f"\n R² on LOG scale: {r2_log:.4f}")
print(f"   Expected: ~0.8683")


 R² on LOG scale: 0.8683
   Expected: ~0.8683


In [357]:
y_pred_log = gb_final.predict(X_test_enc_f)
y_actual_revenue = np.expm1(y_test_rev_f)
y_pred_revenue = np.expm1(y_pred_log)

In [358]:
non_zero_mask = y_actual_revenue >= 10  
actual_errors_pct = abs(y_actual_revenue[non_zero_mask] - y_pred_revenue[non_zero_mask]) / y_actual_revenue[non_zero_mask] * 100

In [359]:
print(f"\nPrediction errors on ACTUAL revenue (excluding < $10M):")
print(f"  Sample size: {sum(non_zero_mask)} predictions")
print(f"  Median error: {np.median(actual_errors_pct):.1f}%")
print(f"  Mean error: {actual_errors_pct.mean():.1f}%")
print(f"  25th percentile: {np.percentile(actual_errors_pct, 25):.1f}%")
print(f"  75th percentile: {np.percentile(actual_errors_pct, 75):.1f}%")


Prediction errors on ACTUAL revenue (excluding < $10M):
  Sample size: 85 predictions
  Median error: 31.3%
  Mean error: 51.9%
  25th percentile: 15.9%
  75th percentile: 61.0%


In [360]:
print(f"\n📊 Error distribution:")
print(f"  < 10% error: {sum(actual_errors_pct < 10)} predictions ({sum(actual_errors_pct < 10)/len(actual_errors_pct)*100:.1f}%)")
print(f"  10-20% error: {sum((actual_errors_pct >= 10) & (actual_errors_pct < 20))} predictions ({sum((actual_errors_pct >= 10) & (actual_errors_pct < 20))/len(actual_errors_pct)*100:.1f}%)")
print(f"  20-30% error: {sum((actual_errors_pct >= 20) & (actual_errors_pct < 30))} predictions ({sum((actual_errors_pct >= 20) & (actual_errors_pct < 30))/len(actual_errors_pct)*100:.1f}%)")
print(f"  30-50% error: {sum((actual_errors_pct >= 30) & (actual_errors_pct < 50))} predictions ({sum((actual_errors_pct >= 30) & (actual_errors_pct < 50))/len(actual_errors_pct)*100:.1f}%)")
print(f"  > 50% error: {sum(actual_errors_pct >= 50)} predictions ({sum(actual_errors_pct >= 50)/len(actual_errors_pct)*100:.1f}%) ⚠️")


📊 Error distribution:
  < 10% error: 10 predictions (11.8%)
  10-20% error: 19 predictions (22.4%)
  20-30% error: 13 predictions (15.3%)
  30-50% error: 14 predictions (16.5%)
  > 50% error: 29 predictions (34.1%) ⚠️


REALISTIC TESTING: COUNTRY-LEVEL PREDICTIONS

In [361]:
country_mapping = {
      'Germany': {
          'Region': 'Europe & Central Asia',
          'Income group': 'High income',
          'Fossil_Fuel_%': 73.0, 
          'Population': 83000000,
          'GDP': 4.4e12  
      },
      'United States': {
          'Region': 'North America',
          'Income group': 'High income',
          'Fossil_Fuel_%': 82.0,
          'Population': 331000000,
          'GDP': 23e12
      },
      'China': {
          'Region': 'East Asia & Pacific',
          'Income group': 'Upper middle income',
          'Fossil_Fuel_%': 85.0,
          'Population': 1400000000,
          'GDP': 17.9e12
      },

      'Pakistan': {
          'Region': 'East Asia & Pacific',  
          'Income group': 'Upper middle income',  
          'Fossil_Fuel_%': 81.2,
          'Population': 247500000,
          'GDP': 1.396e12
      },
      'Brazil': {
          'Region': 'Latin America & Caribbean',
          'Income group': 'Upper middle income',
          'Fossil_Fuel_%': 45.0,  
          'Population': 215000000,
          'GDP': 2.1e12
      },
      'South Africa': {
          'Region': 'Sub-Saharan Africa',
          'Income group': 'Upper middle income',
          'Fossil_Fuel_%': 88.0,
          'Population': 60000000,
          'GDP': 0.4e12
      }
  }

In [362]:
def predict_for_country(country_name, policy_type, carbon_price, year=2025):
      """
      Simulate the full user workflow:
      User selects country → Backend maps to features → Model predicts
      """

      print(f"COUNTRY: {country_name}")
      print(f"POLICY: {policy_type}, ${carbon_price}/ton CO2, Year {year}")

      country_data = country_mapping[country_name]

      test_input = pd.DataFrame({
          'Type': [policy_type],
          'Region': [country_data['Region']],
          'Income group': [country_data['Income group']],
          'Year': [year],
          'Carbon_Price_USD': [carbon_price],
          'Fossil_Fuel_Dependency_%': [country_data['Fossil_Fuel_%']],
          'Population_Log': [np.log1p(country_data['Population'])],
          'GDP': [country_data['GDP']]
      })

      print(f"\nMapped Features:")
      print(f"  Region: {country_data['Region']}")
      print(f"  Income: {country_data['Income group']}")
      print(f"  Fossil Fuel: {country_data['Fossil_Fuel_%']:.1f}%")
      print(f"  Population: {country_data['Population']:,}")
      print(f"  GDP: ${country_data['GDP']/1e12:.2f}T")

 
      if country_data['Region'] not in encoders_direct['Region'].classes_:
          print(f"   WARNING: Region '{country_data['Region']}' not in training data!")
          return None
      if country_data['Income group'] not in encoders_direct['Income group'].classes_:
          print(f"   WARNING: Income group '{country_data['Income group']}' not in training data!")
          return None

      test_encoded = test_input.copy()
      for feature in ['Type', 'Region', 'Income group']:
          test_encoded[feature] = encoders_direct[feature].transform(test_input[feature])

      pred_revenue = gb_direct.predict(test_encoded)[0]
      pred_revenue = max(pred_revenue, 0)  

      
      lower_bound = pred_revenue * 0.70
      upper_bound = pred_revenue * 1.30

      print(f" REVENUE PREDICTION:")
      print(f"  Estimate: ${pred_revenue:,.0f}M/year")
      print(f"  Range: ${lower_bound:,.0f}M - ${upper_bound:,.0f}M")
      print(f"  (±30% confidence interval)")

      
      success_input = test_input[['Type', 'Region', 'Income group', 'Year',
                                  'Fossil_Fuel_Dependency_%', 'GDP']].copy()
      success_encoded = success_input.copy()
      for feature in ['Type', 'Region', 'Income group']:
          success_encoded[feature] = encoders_success[feature].transform(success_input[feature])

      
      abolish_prob = gb_success_final.predict_proba(success_encoded)[0, 0]

      if abolish_prob < 0.35:
          risk = "Low Risk "
      elif abolish_prob < 0.65:
          risk = "At Risk "
      else:
          risk = "High Risk "

      print(f"\n SUCCESS PREDICTION:")
      print(f"  Abolishment Probability: {abolish_prob*100:.1f}%")
      print(f"  Risk Category: {risk}")

      return {
          'revenue': pred_revenue,
          'lower': lower_bound,
          'upper': upper_bound,
          'risk': risk,
          'abolish_prob': abolish_prob
      }

In [363]:
predict_for_country('Pakistan', 'Carbon tax', 15, 2025)

COUNTRY: Pakistan
POLICY: Carbon tax, $15/ton CO2, Year 2025

Mapped Features:
  Region: East Asia & Pacific
  Income: Upper middle income
  Fossil Fuel: 81.2%
  Population: 247,500,000
  GDP: $1.40T
 REVENUE PREDICTION:
  Estimate: $57M/year
  Range: $40M - $74M
  (±30% confidence interval)

 SUCCESS PREDICTION:
  Abolishment Probability: 5.9%
  Risk Category: Low Risk 


{'revenue': 57.13051360453546,
 'lower': 39.99135952317482,
 'upper': 74.2696676858961,
 'risk': 'Low Risk ',
 'abolish_prob': 0.05862022270472267}

In [364]:
predict_for_country('United States', 'Carbon tax', 50, 2025)

COUNTRY: United States
POLICY: Carbon tax, $50/ton CO2, Year 2025

Mapped Features:
  Region: North America
  Income: High income
  Fossil Fuel: 82.0%
  Population: 331,000,000
  GDP: $23.00T
 REVENUE PREDICTION:
  Estimate: $16,638M/year
  Range: $11,647M - $21,630M
  (±30% confidence interval)

 SUCCESS PREDICTION:
  Abolishment Probability: 68.4%
  Risk Category: High Risk 


{'revenue': 16638.370940070185,
 'lower': 11646.859658049128,
 'upper': 21629.88222209124,
 'risk': 'High Risk ',
 'abolish_prob': 0.6836575735460597}

In [365]:
predict_for_country('Germany', 'ETS', 80, 2025)

COUNTRY: Germany
POLICY: ETS, $80/ton CO2, Year 2025

Mapped Features:
  Region: Europe & Central Asia
  Income: High income
  Fossil Fuel: 73.0%
  Population: 83,000,000
  GDP: $4.40T
 REVENUE PREDICTION:
  Estimate: $10,874M/year
  Range: $7,612M - $14,137M
  (±30% confidence interval)

 SUCCESS PREDICTION:
  Abolishment Probability: 2.4%
  Risk Category: Low Risk 


{'revenue': 10874.439993812432,
 'lower': 7612.107995668702,
 'upper': 14136.771991956162,
 'risk': 'Low Risk ',
 'abolish_prob': 0.024425830790227643}

In [366]:
predict_for_country('Brazil', 'Carbon tax', 20, 2025)

COUNTRY: Brazil
POLICY: Carbon tax, $20/ton CO2, Year 2025

Mapped Features:
  Region: Latin America & Caribbean
  Income: Upper middle income
  Fossil Fuel: 45.0%
  Population: 215,000,000
  GDP: $2.10T
 REVENUE PREDICTION:
  Estimate: $1,674M/year
  Range: $1,172M - $2,176M
  (±30% confidence interval)

 SUCCESS PREDICTION:
  Abolishment Probability: 7.0%
  Risk Category: Low Risk 


{'revenue': 1673.6931160012414,
 'lower': 1171.5851812008689,
 'upper': 2175.8010508016137,
 'risk': 'Low Risk ',
 'abolish_prob': 0.06955730700826446}

In [367]:
predict_for_country('China', 'ETS', 30, 2025)

COUNTRY: China
POLICY: ETS, $30/ton CO2, Year 2025

Mapped Features:
  Region: East Asia & Pacific
  Income: Upper middle income
  Fossil Fuel: 85.0%
  Population: 1,400,000,000
  GDP: $17.90T
 REVENUE PREDICTION:
  Estimate: $3,556M/year
  Range: $2,489M - $4,623M
  (±30% confidence interval)

 SUCCESS PREDICTION:
  Abolishment Probability: 2.9%
  Risk Category: Low Risk 


{'revenue': 3556.0022184635313,
 'lower': 2489.201552924472,
 'upper': 4622.802884002591,
 'risk': 'Low Risk ',
 'abolish_prob': 0.029128788444194864}

In [368]:
predict_for_country('South Africa', 'Carbon tax', 10, 2025)

COUNTRY: South Africa
POLICY: Carbon tax, $10/ton CO2, Year 2025

Mapped Features:
  Region: Sub-Saharan Africa
  Income: Upper middle income
  Fossil Fuel: 88.0%
  Population: 60,000,000
  GDP: $0.40T
 REVENUE PREDICTION:
  Estimate: $89M/year
  Range: $63M - $116M
  (±30% confidence interval)

 SUCCESS PREDICTION:
  Abolishment Probability: 1.9%
  Risk Category: Low Risk 


{'revenue': 89.35059524337572,
 'lower': 62.545416670363004,
 'upper': 116.15577381638845,
 'risk': 'Low Risk ',
 'abolish_prob': 0.019169209517269747}

## SUCCESS MODEL COMPREHENSIVE TESTING 

In [369]:
df_complete = pd.read_csv('../dataset/processed/ecoimpact_complete_dataset.csv')

GET LIST OF JURISDICTIONS IN TRAINING DATA

In [370]:
print(f"Total rows in dataset: {len(df_complete)}")
print(f"Unique jurisdictions: {df_complete['Jurisdiction'].nunique()}")
print(f"Year range: {df_complete['Year'].min()} - {df_complete['Year'].max()}")

Total rows in dataset: 2808
Unique jurisdictions: 68
Year range: 1990 - 2025


In [371]:
required_cols = ['Type', 'Region', 'Income group', 'Year', 'Fossil_Fuel_Dependency_%', 'GDP', 'Status']

In [372]:
df_with_features = df_complete[required_cols + ['Jurisdiction']].copy()

In [373]:
df_complete_data = df_with_features.dropna()

In [374]:
latest_data = df_complete_data.sort_values('Year').groupby('Jurisdiction').last().reset_index()

In [375]:
print(f"Testing {len(latest_data)} jurisdictions")
print(f"Year range of latest data: {latest_data['Year'].min()} - {latest_data['Year'].max()}")

Testing 67 jurisdictions
Year range of latest data: 2023 - 2023


In [376]:
test_features = latest_data[['Type', 'Region', 'Income group', 'Year', 'Fossil_Fuel_Dependency_%', 'GDP']].copy()

In [377]:
test_encoded = test_features.copy()
for col in ['Type', 'Region', 'Income group']:
      test_encoded[col] = encoders_success[col].transform(test_features[col])

In [ ]:
abolishment_probs = gb_success_final.predict_proba(test_encoded)[:, 0]
abolishment_risk_pct = abolishment_probs * 100

In [ ]:
def categorize_risk(prob):
      if prob < 0.35:
          return "Low Risk"
      elif prob < 0.65:
          return "At Risk"
      else:
          return "High Risk"

risk_categories = [categorize_risk(p) for p in abolishment_probs]

In [ ]:
results = pd.DataFrame({
      'Jurisdiction': latest_data['Jurisdiction'],
      'Year': latest_data['Year'],
      'Type': latest_data['Type'],
      'Region': latest_data['Region'],
      'Income_Group': latest_data['Income group'],
      'Abolishment_Risk_%': abolishment_risk_pct.round(1),
      'Risk_Category': risk_categories,
      'Status': latest_data['Status']
  })

In [ ]:
results_sorted = results.sort_values('Abolishment_Risk_%', ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 30)

In [ ]:
print(results_sorted.to_string(index=False))

               Jurisdiction  Year       Type                     Region        Income_Group  Abolishment_Risk_% Risk_Category      Status
                    Alberta  2023 Carbon tax              North America         High income                88.3     High Risk   Abolished
       Prince Edward Island  2023 Carbon tax              North America         High income                88.3     High Risk   Abolished
           British Columbia  2023 Carbon tax              North America         High income                88.3     High Risk   Abolished
                     Canada  2023 Carbon tax              North America         High income                88.3     High Risk   Abolished
      Northwest Territories  2023 Carbon tax              North America         High income                88.3     High Risk   Abolished
  Newfoundland and Labrador  2023 Carbon tax              North America         High income                88.3     High Risk   Abolished
              New Brunswick  2023 

## Saving Models

In [ ]:
import pickle
import json
from datetime import datetime

Revenue Model - gb_direct with encoders_direct

In [ ]:
with open('../ML Model/revenue_model_gb.pkl', 'wb') as f:
      pickle.dump(gb_direct, f)

In [ ]:
with open('../ML Model/revenue_encoders.pkl', 'wb') as f:
      pickle.dump(encoders_direct, f)

Success Model - gb_success_final with encoders_success

In [ ]:
with open('../ML Model/success_model_gb.pkl', 'wb') as f:
      pickle.dump(gb_success_final, f)

In [ ]:
with open('../ML Model/success_encoders.pkl', 'wb') as f:
      pickle.dump(encoders_success, f)

Metadata

In [ ]:
metadata = {
      "saved_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
      "revenue_model": {
          "variable_name": "gb_direct",
          "encoders_variable": "encoders_direct",
          "algorithm": "GradientBoostingRegressor",
          "loss": "huber",
          "test_r2": 0.8676,
          "median_error_percent": 26.8,
          "features": ["Type", "Region", "Income group", "Year", "Carbon_Price_USD", "Fossil_Fuel_Dependency_%", "Population_Log", "GDP"],    
          "target": "Revenue_Million_USD",
          "n_features": 8,
          "hyperparameters_note": "Stored in model object"
      },
      "success_model": {
          "variable_name": "gb_success_final",
          "encoders_variable": "encoders_success",
          "algorithm": "GradientBoostingClassifier",
          "test_accuracy": 0.954,
          "test_precision": 1.00,
          "test_recall": 0.675,
          "features": ["Type", "Region", "Income group", "Year", "Fossil_Fuel_Dependency_%", "GDP"],
          "target": "Status",
          "n_features": 6,
          "risk_thresholds": {"low": 0.35, "high": 0.65},
          "hyperparameters_note": "Stored in model object (from GridSearchCV best_estimator_)"
      }
  }

In [ ]:
with open('../ML Model/model_metadata.json', 'w') as f:
      json.dump(metadata, f, indent=2)

In [ ]:
print("Models saved")

Models saved


## Model Enhancement: Retraining with Coverage Feature using NEW COVERAGE DATASET

##### Problem Identified (Dec 29, 2024)

  **Original Models:**
  - Revenue Model: 8 features (Type, Region, Income, Year, Price, Fossil%, Pop_Log, GDP)
  - Success Model: 6 features (Type, Region, Income, Year, Fossil%, GDP)
  - **Coverage was NOT a feature**

  **Issue:**
  - Training data had coverage values of 0.07-4.27% (very low variance, std=0.01%)
  - Cannot predict scenarios at 20-80% coverage (100-500x extrapolation)
  - Users want to simulate different coverage levels, but model can't handle it

### Load Enhanced Data

In [ ]:
df_enhanced = pd.read_csv(r'C:\Users\HP\Desktop\fyp\docs\Ecoimpact -AI\dataset\processed\ecoimpact_clean_for_retraining.csv')

In [ ]:
print(f"\nDataset: {len(df_enhanced):,} rows")
print(f"Jurisdictions: {df_enhanced['Jurisdiction'].nunique()}")
print(f"Year range: {df_enhanced['Year'].min()} - {df_enhanced['Year'].max()}")


Dataset: 548 rows
Jurisdictions: 56
Year range: 1994 - 2023


In [ ]:
print(f"Coverage Statistics:")
print(f"  Min:    {df_enhanced['Actual_Coverage_%'].min():.2f}%")
print(f"  Max:    {df_enhanced['Actual_Coverage_%'].max():.2f}%")
print(f"  Mean:   {df_enhanced['Actual_Coverage_%'].mean():.2f}%")
print(f"  Median: {df_enhanced['Actual_Coverage_%'].median():.2f}%")
print(f"  Std:    {df_enhanced['Actual_Coverage_%'].std():.2f}%")

Coverage Statistics:
  Min:    1.03%
  Max:    96.10%
  Mean:   34.73%
  Median: 36.47%
  Std:    24.45%


In [ ]:
df_enhanced['Coverage_x_GDP'] = df_enhanced['Actual_Coverage_%'] * df_enhanced['GDP']

In [ ]:
revenue_features_coverage = [
      'Type', 'Region', 'Income group', 'Year',
      'Carbon_Price_USD', 'Actual_Coverage_%',
      'Coverage_x_GDP',  
      'Fossil_Fuel_Dependency_%', 'Population_Log', 'GDP'
  ]

In [ ]:
df_enhanced['Population_Log'] = np.log(df_enhanced['Population'])

In [ ]:
df_revenue_coverage = df_enhanced[
      revenue_features_coverage + ['Revenue_Million_USD']
  ].dropna().copy()

In [ ]:
print(f"Target: Revenue_Million_USD")
print(f"  Min:    ${df_revenue_coverage['Revenue_Million_USD'].min():.1f}M")
print(f"  Max:    ${df_revenue_coverage['Revenue_Million_USD'].max():.1f}M")
print(f"  Mean:   ${df_revenue_coverage['Revenue_Million_USD'].mean():.1f}M")

Target: Revenue_Million_USD
  Min:    $0.0M
  Max:    $11670.2M
  Mean:   $780.3M


In [ ]:
X_revenue_cov = df_revenue_coverage[revenue_features_coverage]
y_revenue_cov = df_revenue_coverage['Revenue_Million_USD']

In [ ]:
X_train_rev_cov, X_test_rev_cov, y_train_rev_cov, y_test_rev_cov = train_test_split(
      X_revenue_cov, y_revenue_cov,
      test_size=0.2,
      random_state=42,
      stratify=X_revenue_cov['Region']
  )

In [ ]:
encoders_revenue_cov = {}
X_train_rev_cov_enc = X_train_rev_cov.copy()
X_test_rev_cov_enc = X_test_rev_cov.copy()

In [ ]:
categorical_features_rev = ['Type', 'Region', 'Income group']

In [ ]:
for feature in categorical_features_rev:
      encoders_revenue_cov[feature] = LabelEncoder()
      X_train_rev_cov_enc[feature] = encoders_revenue_cov[feature].fit_transform(X_train_rev_cov[feature])
      X_test_rev_cov_enc[feature] = encoders_revenue_cov[feature].transform(X_test_rev_cov[feature])

In [ ]:
print("Categorical features encoded:")
for feature in categorical_features_rev:
    print(f"  {feature}: {len(encoders_revenue_cov[feature].classes_)} classes")

Categorical features encoded:
  Type: 2 classes
  Region: 5 classes
  Income group: 2 classes


In [ ]:
gb_revenue_cov = GradientBoostingRegressor(
      loss='huber',
      alpha=0.9,
      learning_rate=0.01,
      max_depth=5,
      min_samples_split=5,
      n_estimators=500,
      subsample=0.8,
      random_state=42
  )

In [ ]:
gb_revenue_cov.fit(X_train_rev_cov_enc, y_train_rev_cov)

GradientBoostingRegressor(learning_rate=0.01, loss='huber', max_depth=5,
                          min_samples_split=5, n_estimators=500,
                          random_state=42, subsample=0.8)

In [ ]:
gb_revenue_grid = GradientBoostingRegressor(random_state=42)

In [ ]:
param_grid_revenue = {
      'loss': ['huber'],
      'alpha': [0.9],
      'learning_rate': [0.01, 0.05, 0.1],
      'max_depth': [3, 5, 7],
      'min_samples_split': [2, 5, 10],
      'n_estimators': [100, 200, 300, 500],
      'subsample': [0.8, 1.0]
  }

In [ ]:
grid_search_revenue_cov = GridSearchCV(
      gb_revenue_grid,
      param_grid_revenue,
      cv=5,
      scoring='r2',
      n_jobs=-1,
      verbose=1
  )

In [ ]:
grid_search_revenue_cov.fit(X_train_rev_cov_enc, y_train_rev_cov)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
             n_jobs=-1,
             param_grid={'alpha': [0.9], 'learning_rate': [0.01, 0.05, 0.1],
                         'loss': ['huber'], 'max_depth': [3, 5, 7],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300, 500],
                         'subsample': [0.8, 1.0]},
             scoring='r2', verbose=1)

In [ ]:
print(f"Best parameters:")
for param, value in grid_search_revenue_cov.best_params_.items():

    print(f"  {param}: {value}")

Best parameters:
  alpha: 0.9
  learning_rate: 0.01
  loss: huber
  max_depth: 5
  min_samples_split: 5
  n_estimators: 500
  subsample: 0.8


In [ ]:
gb_revenue_cov = grid_search_revenue_cov.best_estimator_

In [ ]:
y_pred_train_rev_cov = gb_revenue_cov.predict(X_train_rev_cov_enc)
y_pred_test_rev_cov = gb_revenue_cov.predict(X_test_rev_cov_enc)

In [ ]:
y_pred_train_rev_cov = np.maximum(y_pred_train_rev_cov, 0)
y_pred_test_rev_cov = np.maximum(y_pred_test_rev_cov, 0)

In [ ]:
train_r2_cov = r2_score(y_train_rev_cov, y_pred_train_rev_cov)
test_r2_cov = r2_score(y_test_rev_cov, y_pred_test_rev_cov)
test_mae_cov = mean_absolute_error(y_test_rev_cov, y_pred_test_rev_cov)

In [ ]:
errors_pct_cov = np.abs(y_test_rev_cov - y_pred_test_rev_cov) / (y_test_rev_cov + 1) * 100
median_error_pct_cov = np.median(errors_pct_cov)

In [ ]:
print(f"\nTrain R²: {train_r2_cov:.4f}")
print(f"Test R²:  {test_r2_cov:.4f}")
print(f"Test MAE: ${test_mae_cov:.1f}M")
print(f"Median Error: {median_error_pct_cov:.1f}%")


Train R²: 0.8760
Test R²:  0.8899
Test MAE: $188.0M
Median Error: 49.3%


In [ ]:
feature_importance_cov = pd.DataFrame({
      'feature': revenue_features_coverage,
      'importance': gb_revenue_cov.feature_importances_
  }).sort_values('importance', ascending=False)

In [ ]:
coverage_rank = feature_importance_cov[feature_importance_cov['feature'] == 'Actual_Coverage_%'].index[0] + 1
coverage_importance = feature_importance_cov[feature_importance_cov['feature'] == 'Actual_Coverage_%']['importance'].values[0]

In [ ]:
print(f"\nActual_Coverage_% ranked #{coverage_rank} out of 9 features")
print(f"Importance score: {coverage_importance:.4f}")

if coverage_rank <= 3:
    print(f"Coverage is a TOP 3 feature - highly influential!")
elif coverage_rank <= 5:
    print(f" Coverage is moderately influential")
else:
    print(f" Coverage has lower importance - model may rely on other features")


Actual_Coverage_% ranked #6 out of 9 features
Importance score: 0.0709
 Coverage has lower importance - model may rely on other features


In [ ]:
print(f"\nTest set revenue distribution:")
print(f"  Min:    ${y_test_rev_cov.min():.1f}M")
print(f"  Max:    ${y_test_rev_cov.max():.1f}M")
print(f"  Mean:   ${y_test_rev_cov.mean():.1f}M")
print(f"  Median: ${y_test_rev_cov.median():.1f}M")


Test set revenue distribution:
  Min:    $0.0M
  Max:    $8537.2M
  Mean:   $601.0M
  Median: $120.7M


In [ ]:
print(f"\nPredictions:")
print(f"  Min:    ${y_pred_test_rev_cov.min():.1f}M")
print(f"  Max:    ${y_pred_test_rev_cov.max():.1f}M")
print(f"  Mean:   ${y_pred_test_rev_cov.mean():.1f}M")


Predictions:
  Min:    $0.0M
  Max:    $6130.9M
  Mean:   $563.9M


In [ ]:
small_revenue = y_test_rev_cov < 100  
large_revenue = y_test_rev_cov >= 100

print(f"\nError by revenue size:")
print(f"  Small (<$100M): {small_revenue.sum()} samples, Median Error: {np.median(errors_pct_cov[small_revenue]):.1f}%")
print(f"  Large (≥$100M): {large_revenue.sum()} samples, Median Error: {np.median(errors_pct_cov[large_revenue]):.1f}%")


print(f"\nSample predictions (first 10):")
comparison = pd.DataFrame({
      'Actual': y_test_rev_cov.values[:10],
      'Predicted': y_pred_test_rev_cov[:10],
      'Error_%': errors_pct_cov[:10]
  })
print(comparison.to_string(index=False))


Error by revenue size:
  Small (<$100M): 54 samples, Median Error: 393.8%
  Large (≥$100M): 56 samples, Median Error: 18.6%

Sample predictions (first 10):
     Actual   Predicted      Error_%
 349.354271  373.701199     6.949231
 564.715452  818.625413    44.882981
 433.191200  437.178820     0.918402
1453.760000 1545.759376     6.324024
   0.000000  479.331354 47933.135375
 156.117113   46.216756    69.948050
4260.739510 3788.684379    11.076583
   0.000000    2.199039   219.903904
   0.140332    2.590095   214.828857
   0.000000   10.013415  1001.341500


In [ ]:
meaningful_revenue = y_test_rev_cov >= 10 

In [ ]:
errors_pct_meaningful = np.abs(
      y_test_rev_cov[meaningful_revenue] - y_pred_test_rev_cov[meaningful_revenue]
  ) / y_test_rev_cov[meaningful_revenue] * 100

median_error_meaningful = np.median(errors_pct_meaningful)

In [ ]:
print(f"  Median Error: {median_error_meaningful:.1f}%")

  Median Error: 24.1%


In [ ]:
test_low_cov = pd.DataFrame({
      'Type': ['Carbon tax'],
      'Region': ['East Asia & Pacific'],
      'Income group': ['Upper middle income'],
      'Year': [2025],
      'Carbon_Price_USD': [25.0],
      'Actual_Coverage_%': [20.0],  # LOW coverage
      'Coverage_x_GDP': [20.0 * 50000],  # Coverage × GDP
      'Fossil_Fuel_Dependency_%': [70.0],
      'Population_Log': [np.log(220_000_000)],
      'GDP': [50000]
  })

test_high_cov = pd.DataFrame({
      'Type': ['Carbon tax'],
      'Region': ['East Asia & Pacific'],
      'Income group': ['Upper middle income'],
      'Year': [2025],
      'Carbon_Price_USD': [25.0],
      'Actual_Coverage_%': [80.0],  # HIGH coverage
      'Coverage_x_GDP': [80.0 * 50000],  # Coverage × GDP
      'Fossil_Fuel_Dependency_%': [70.0],
      'Population_Log': [np.log(220_000_000)],
      'GDP': [50000]
  })

In [ ]:
test_low_enc = test_low_cov.copy()
test_high_enc = test_high_cov.copy()

In [ ]:
for feature in categorical_features_rev:
      test_low_enc[feature] = encoders_revenue_cov[feature].transform(test_low_cov[feature])
      test_high_enc[feature] = encoders_revenue_cov[feature].transform(test_high_cov[feature])

In [ ]:
test_low_enc = test_low_enc[revenue_features_coverage]
test_high_enc = test_high_enc[revenue_features_coverage]

In [ ]:
pred_low = gb_revenue_cov.predict(test_low_enc)[0]
pred_high = gb_revenue_cov.predict(test_high_enc)[0]

In [ ]:
print(f"\nTest: Pakistan, $25/tonne Carbon Tax")
print(f"\nScenario 1 - 20% coverage:")
print(f"  Predicted Revenue: ${pred_low:.1f}M")

print(f"\nScenario 2 - 80% coverage:")
print(f"  Predicted Revenue: ${pred_high:.1f}M")

print(f"\nDifference: ${pred_high - pred_low:.1f}M ({(pred_high/pred_low - 1)*100:.1f}% increase)")




Test: Pakistan, $25/tonne Carbon Tax

Scenario 1 - 20% coverage:
  Predicted Revenue: $279.7M

Scenario 2 - 80% coverage:
  Predicted Revenue: $402.6M

Difference: $122.9M (44.0% increase)


In [ ]:
correlation = df_revenue_coverage[['Actual_Coverage_%', 'Revenue_Million_USD']].corr()
print(f"\nCorrelation in training data:")
print(correlation)


Correlation in training data:
                     Actual_Coverage_%  Revenue_Million_USD
Actual_Coverage_%             1.000000             0.281475
Revenue_Million_USD           0.281475             1.000000


In [ ]:
print(f"\nRevenue by Coverage quartiles:")
df_revenue_coverage['Coverage_Quartile'] = pd.qcut(df_revenue_coverage['Actual_Coverage_%'], 4, labels=['Q1 (Low)', 'Q2', 'Q3', 'Q4 (High)'])
quartile_stats = df_revenue_coverage.groupby('Coverage_Quartile').agg({
      'Revenue_Million_USD': ['count', 'mean', 'median'],
      'Actual_Coverage_%': 'mean',
      'Carbon_Price_USD': 'mean',
      'GDP': 'mean'
  }).round(2)
print(quartile_stats)


Revenue by Coverage quartiles:
                  Revenue_Million_USD                  Actual_Coverage_% Carbon_Price_USD           GDP
                                count     mean  median              mean             mean          mean
Coverage_Quartile                                                                                      
Q1 (Low)                          143   326.36    4.77              6.04             9.02  1.150113e+13
Q2                                133   652.33  140.70             21.30            29.60  4.514775e+12
Q3                                135   639.80  211.27             46.77            38.36  5.791987e+12
Q4 (High)                         137  1516.76  507.12             65.83            35.57  1.341695e+12


In [ ]:
region_coverage = df_revenue_coverage.groupby('Region')['Actual_Coverage_%'].mean().sort_values(ascending=False)
print(region_coverage)

Region
Sub-Saharan Africa           93.106827
Latin America & Caribbean    43.871878
Europe & Central Asia        40.367332
East Asia & Pacific          36.535420
North America                15.123662
Name: Actual_Coverage_%, dtype: float64


In [ ]:
type_stats = df_revenue_coverage.groupby('Type').agg({
      'Actual_Coverage_%': 'mean',
      'Revenue_Million_USD': 'mean'
  }).round(2)
print(type_stats)

            Actual_Coverage_%  Revenue_Million_USD
Type                                              
Carbon tax              41.72               962.00
ETS                     23.20               480.95


## SUCCESS MODEL WITH COVERAGE FEATURE

In [ ]:
success_features_coverage = [
      'Type', 'Region', 'Income group', 'Year',
      'Actual_Coverage_%',
      'Fossil_Fuel_Dependency_%', 'GDP'
  ]

In [ ]:
df_success_coverage = df_enhanced[
      (df_enhanced['Status'].isin(['Implemented', 'Abolished'])) &
      (df_enhanced[success_features_coverage].notna().all(axis=1))
  ].copy()

In [ ]:
df_success_coverage['Status_Binary'] = (df_success_coverage['Status'] == 'Abolished').astype(int)

In [ ]:
X_success_cov = df_success_coverage[success_features_coverage]
y_success_cov = df_success_coverage['Status_Binary']

In [ ]:
X_train_suc_cov, X_test_suc_cov, y_train_suc_cov, y_test_suc_cov = train_test_split(
      X_success_cov, y_success_cov,
      test_size=0.2,
      random_state=42,
      stratify=X_success_cov['Region']
  )

In [ ]:
encoders_success_cov = {}
X_train_suc_cov_enc = X_train_suc_cov.copy()
X_test_suc_cov_enc = X_test_suc_cov.copy()

categorical_features_suc = ['Type', 'Region', 'Income group']
for feature in categorical_features_suc:
      encoders_success_cov[feature] = LabelEncoder()
      X_train_suc_cov_enc[feature] = encoders_success_cov[feature].fit_transform(X_train_suc_cov[feature])
      X_test_suc_cov_enc[feature] = encoders_success_cov[feature].transform(X_test_suc_cov[feature])

In [ ]:
print("Categorical features encoded:")
for feature in categorical_features_suc:
    print(f"  {feature}: {len(encoders_success_cov[feature].classes_)} classes")

Categorical features encoded:
  Type: 2 classes
  Region: 5 classes
  Income group: 2 classes


In [ ]:
gb_success_grid = GradientBoostingClassifier(random_state=42)

In [ ]:
param_grid_success = {
      'learning_rate': [0.01, 0.05, 0.1],
      'max_depth': [3, 5, 7],
      'min_samples_split': [2, 5, 10],
      'n_estimators': [100, 200, 300],
      'subsample': [0.8, 1.0]
  }

In [ ]:
grid_search_success_cov = GridSearchCV(
      gb_success_grid,
      param_grid_success,
      cv=5,
      scoring='accuracy',
      n_jobs=-1,
      verbose=1
  )

In [ ]:
grid_search_success_cov.fit(X_train_suc_cov_enc, y_train_suc_cov)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=1)

In [ ]:
for param, value in grid_search_success_cov.best_params_.items():
      print(f"  {param}: {value}")

print(f"Best CV accuracy: {grid_search_success_cov.best_score_:.4f}")

  learning_rate: 0.01
  max_depth: 3
  min_samples_split: 2
  n_estimators: 100
  subsample: 0.8
Best CV accuracy: 0.9909


In [ ]:
gb_success_cov = grid_search_success_cov.best_estimator_

NameError: name 'grid_search_success_cov' is not defined

In [ ]:
y_pred_suc_cov = gb_success_cov.predict(X_test_suc_cov_enc)
y_pred_proba_suc_cov = gb_success_cov.predict_proba(X_test_suc_cov_enc)[:, 1]

In [ ]:
accuracy_cov = accuracy_score(y_test_suc_cov, y_pred_suc_cov)
precision_cov = precision_score(y_test_suc_cov, y_pred_suc_cov, zero_division=0)
recall_cov = recall_score(y_test_suc_cov, y_pred_suc_cov, zero_division=0)
f1_cov = f1_score(y_test_suc_cov, y_pred_suc_cov, zero_division=0)

In [ ]:
print(f"Test Set Metrics:")
print(f"  Accuracy:  {accuracy_cov:.4f}")
print(f"  Precision: {precision_cov:.4f}")
print(f"  Recall:    {recall_cov:.4f}")
print(f"  F1-Score:  {f1_cov:.4f}")


Test Set Metrics:
  Accuracy:  0.9818
  Precision: 1.0000
  Recall:    0.7778
  F1-Score:  0.8750


In [ ]:
feature_importance_suc_cov = pd.DataFrame({
      'feature': success_features_coverage,
      'importance': gb_success_cov.feature_importances_
  }).sort_values('importance', ascending=False)

In [ ]:
print("\nRanked by importance:")
for i, (_, row) in enumerate(feature_importance_suc_cov.iterrows(), 1):
    bar = '█' * int(row['importance'] * 100)
    print(f"{i}. {row['feature']:30s} {row['importance']:.4f} {bar}")

coverage_rank = list(feature_importance_suc_cov['feature']).index('Actual_Coverage_%') + 1
coverage_importance = feature_importance_suc_cov[feature_importance_suc_cov['feature'] == 'Actual_Coverage_%']['importance'].values[0]      

print(f"\nActual_Coverage_% ranked #{coverage_rank} with importance {coverage_importance:.4f}")


Ranked by importance:
1. Type                           0.5498 ██████████████████████████████████████████████████████
2. Region                         0.3518 ███████████████████████████████████
3. Actual_Coverage_%              0.0341 ███
4. GDP                            0.0340 ███
5. Fossil_Fuel_Dependency_%       0.0193 █
6. Income group                   0.0105 █
7. Year                           0.0006 

Actual_Coverage_% ranked #3 with importance 0.0341


##### Summary: Both Models Retrained Successfully

  ###### Revenue Model
  - **Features:** 10 (added Coverage_x_GDP interaction)
  - **Test R²:** 0.8899 (improved from 0.8676)
  - **Median Error:** 24.1% (improved from 26.8%)
  - **Coverage Effect:** ✓ Positive (higher coverage → higher revenue)

  ##### Success Model
  - **Features:** 7 (added Actual_Coverage_%)
  - **Accuracy:** 98.2% (improved from 95.4%)
  - **Precision:** 100% (maintained)
  - **Recall:** 77.8% (improved from 67.5%)

  Both models now handle 20-80% coverage scenarios without extrapolation!

## Saving IMPROVED MODELS

In [382]:
import joblib
import json
from datetime import datetime
from pathlib import Path

In [ ]:
models_dir = Path(r'C:\Users\HP\Desktop\fyp\docs\Ecoimpact -AI\ML Model')

In [ ]:
joblib.dump(gb_revenue_cov, models_dir / 'revenue_model_gb.pkl')
joblib.dump(encoders_revenue_cov, models_dir / 'revenue_encoders.pkl')
print(f"\n Revenue Model Saved:")
print(f"  - revenue_model_gb.pkl")
print(f"  - revenue_encoders.pkl")
print(f"  Features: {len(revenue_features_coverage)}")
print(f"  Test R²: {test_r2_cov:.4f}")


 Revenue Model Saved:
  - revenue_model_gb.pkl
  - revenue_encoders.pkl
  Features: 10
  Test R²: 0.8899


In [ ]:
joblib.dump(gb_success_cov, models_dir / 'success_model_gb.pkl')
joblib.dump(encoders_success_cov, models_dir / 'success_encoders.pkl')
print(f"\n Success Model Saved:")
print(f"  - success_model_gb.pkl")
print(f"  - success_encoders.pkl")
print(f"  Features: {len(success_features_coverage)}")
print(f"  Accuracy: {accuracy_cov:.4f}")


 Success Model Saved:
  - success_model_gb.pkl
  - success_encoders.pkl
  Features: 7
  Accuracy: 0.9818


In [ ]:
metadata = {
      "saved_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
      "revenue_model": {
          "variable_name": "gb_revenue_cov",
          "encoders_variable": "encoders_revenue_cov",
          "algorithm": "GradientBoostingRegressor",
          "loss": "huber",
          "test_r2": round(test_r2_cov, 4),
          "median_error_percent": 24.1,
          "features": revenue_features_coverage,
          "target": "Revenue_Million_USD",
          "n_features": len(revenue_features_coverage),
          "coverage_included": True,
          "coverage_range": "1.03% - 96.10%",
          "hyperparameters": grid_search_revenue_cov.best_params_
      },
      "success_model": {
          "variable_name": "gb_success_cov",
          "encoders_variable": "encoders_success_cov",
          "algorithm": "GradientBoostingClassifier",
          "test_accuracy": round(accuracy_cov, 4),
          "test_precision": round(precision_cov, 4),
          "test_recall": round(recall_cov, 4),
          "features": success_features_coverage,
          "target": "Status",
          "n_features": len(success_features_coverage),
          "coverage_included": True,
          "risk_thresholds": {"low": 0.35, "high": 0.65},
          "hyperparameters": grid_search_success_cov.best_params_
      }
  }

In [ ]:
with open(models_dir / 'model_metadata.json', 'w') as f:
      json.dump(metadata, f, indent=2)

saving gb_Success_final again as gb_succes_cov resulted in overfitting

In [379]:
with open('../ML Model/success_model_gb.pkl', 'wb') as f:
      pickle.dump(gb_success_final, f)

In [380]:
with open('../ML Model/success_encoders.pkl', 'wb') as f:
      pickle.dump(encoders_success, f)

updated metadata

In [383]:
metadata = {
      "saved_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
      "revenue_model": {
          "variable_name": "gb_revenue_cov",
          "encoders_variable": "encoders_revenue_cov",
          "algorithm": "GradientBoostingRegressor",
          "loss": "huber",
          "test_r2": 0.8899,
          "median_error_percent": 24.1,
          "features": ["Type", "Region", "Income group", "Year", "Carbon_Price_USD", "Actual_Coverage_%", "Coverage_x_GDP", "Fossil_Fuel_Dependency_%", "Population_Log", "GDP"],
          "target": "Revenue_Million_USD",
          "n_features": 10,
          "coverage_included": True,
          "coverage_range": "1.03% - 96.10%",
          "hyperparameters": {
              "alpha": 0.9,
              "learning_rate": 0.01,
              "loss": "huber",
              "max_depth": 5,
              "min_samples_split": 5,
              "n_estimators": 500,
              "subsample": 0.8
          }
      },
      "success_model": {
          "variable_name": "gb_success_final",
          "encoders_variable": "encoders_success",
          "algorithm": "GradientBoostingClassifier",
          "test_accuracy": 0.954,
          "test_precision": 1.00,
          "test_recall": 0.675,
          "features": ["Type", "Region", "Income group", "Year", "Fossil_Fuel_Dependency_%", "GDP"],
          "target": "Status",
          "n_features": 6,
          "coverage_included": False,
          "risk_thresholds": {"low": 0.35, "high": 0.65},
          "hyperparameters_note": "Stored in model object (from GridSearchCV best_estimator_)"
      }
  }

In [385]:
with open('../ML Model/model_metadata.json', 'w') as f:
      json.dump(metadata, f, indent=2)